In [1]:
import numpy as np
from numpy import *
import pandas as pd
import networkx as nx
import pylab
import turtle
import cv2
import random
import time
import sys
import PIL
from collections import deque
from itertools import combinations,groupby
import time
import string
from pygsp import graphs, filters
import re
import networkx as nx
from importlib.metadata import version
from datetime import datetime
from datetime import timedelta
import pandas 
import os

import PySide6
from PySide6 import QtUiTools,QtWidgets
from PySide6.QtGui import *
from PySide6.QtWidgets import QApplication,QMainWindow
from PySide6 import QtCore

import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg
from matplotlib.figure import Figure

import wntr
import tsnet
#import pandapipes as pp
from epyt import epanet

#from ui_MainWindow import ui_MainWindow
import ui_MainWindow
import UTwinRoutine as utr

In [3]:
#check versions
print("NetworkX version - " + str(version('networkx')))
print("WNTR version - "+str(version('wntr')))
print("TSNet version - "+str(version('tsnet')))

NetworkX version - 3.3
WNTR version - 1.2.0
TSNet version - 0.3.1


In [5]:
#links
#GRAPH for water resources vizualization
#https://www.theoj.org/joss-papers/joss.05139/10.21105.joss.05139.pdf
#https://waterprogramming.wordpress.com/tag/networkx/

In [7]:
#additional classes
class MplCanvas(FigureCanvasQTAgg):
    def __init__(self, parent=None, width=5, height=4, dpi=100):
        self.fig = Figure(dpi=dpi)#figsize=(width, height), dpi=dpi)
        self.axis = self.fig.add_subplot(111)
        super(MplCanvas, self).__init__(self.fig)

    def plot_signal(self,x,y):
        self.axis.clear()
        self.fig.canvas.draw()   
        self.axis.plot(x,y)  
        self.fig.canvas.draw()        
        plt.show()

    def UpdateFigure(self):
        #https://stackoverflow.com/questions/56348302/how-to-plot-graphs-iteratively-on-jupyter-notebook
        ani = animation.FuncAnimation(fig, update, frames=edges, interval=1000., init_func=init, repeat=False)

    def show_graph(self,Graph,pos2,node_color='wheat', node_size=50,with_labels=False):
        self.axis.clear()
        self.fig.canvas.draw()   
        nx.draw(Graph, pos=pos2, ax=self.axis,node_color=node_color, node_size=node_size,with_labels=with_labels)
        self.fig.canvas.draw()        
        plt.show()
        #nx.draw(Graph, pos=pos2, node_color='wheat', node_size=50,with_labels=False)
        #pylab.show()
    def highlight_element(self,Graph,pos2,
                          highlight_name="",
                          node_color='wheat',
                          node_size=50,
                          with_labels=False,
                          highlight_color="red",
                          highlight_labels_size=6,
                          highlight_labels_color="green"
                          ):
        # draw graph
        self.axis.clear()
        self.fig.canvas.draw()   
        nx.draw(Graph, pos=pos2, ax=self.axis,node_color=node_color, node_size=node_size,with_labels=with_labels)
        #now we highlight
        if(highlight_name!=""):            
            #first, pass through all nodes
            #highlight_graph=nx.DiGraph(directed=True,strict=True)
            
            Nodes_highlight=False
            for outer_d in Graph.nodes(data=True):
                #print(outer_d[0])
                name_cur=outer_d[0]
                if(outer_d==highlight_name):                    
                    #highlight_graph.add_node(outer_d)
                    break
            if(Nodes_highlight==True):
                # draw subgraph for highlights
                nx.draw_networkx(Graph.subgraph(highlight_name), pos=pos2, node_size = node_size,
                                                                            font_size=highlight_labels_size, 
                                                                            node_color=highlight_color, 
                                                                            font_color=highlight_labels_color,
                                                                            ax=self.axis,
                                                                            
                                                        )
                print("Selected node: "+str(highlight_name))
                Nodes_highlight==True

            #if we did not fiund any nodes we find the conduits
            if(Nodes_highlight==False):#we did not find nodes, now we go to edges
                #tmpList = re.split('-', highlight_name)
                for edge in list(Graph.edges(data=True)):
                    
                    edge_name_1=edge[0]
                    edge_name_2=edge[1]     
                    
                    if((edge_name_1 in highlight_name) and (edge_name_2 in highlight_name)):#highlight_name[0]==edge_name):   

                        # draw subgraph for highlights
                        nx.draw_networkx(Graph.subgraph(edge_name_1), pos=pos2, 
                                                                            node_size = node_size,
                                                                            font_size=highlight_labels_size, 
                                                                            node_color=highlight_color, 
                                                                            font_color=highlight_labels_color,
                                                                            ax=self.axis,
                                                                            
                                                        )
                        
                        nx.draw_networkx(Graph.subgraph(edge_name_2), pos=pos2, 
                                                                            font_size=highlight_labels_size, 
                                                                            node_color=highlight_color, 
                                                                            font_color=highlight_labels_color,
                                                                            ax=self.axis,
                                                                            
                                                        )
                        """
                        #highlight_graph.add_edge(Graph.edges[highlight_name])#edge_name_1,edge_name_2)                    
                        position={} 
                        FlagEdge_1=False
                        FlagEdge_2=False
                        
                        pos_node_1 = np.asarray((float(Graph.nodes[edge_name_1]["X"]),float(Graph.nodes[edge_name_1]["Y"])))  
                        if(pos_node_1 is not None):
                            position[edge_name_1] =    pos_node_1    
                            highlight_graph.add_nodes_from(Graph.nodes[edge_name_1])#add_node(Graph.nodes[edge_name_1])#edge_name_1)
                            #print(Graph.nodes[edge_name_1])
                            FlagEdge_1=True

                        pos_node_2 = np.asarray((float(Graph.nodes[edge_name_2]["X"]),float(Graph.nodes[edge_name_2]["Y"])))  
                        if(pos_node_2 is not None):
                            position[edge_name_2] =  pos_node_2
                            highlight_graph.add_nodes_from(Graph.nodes[edge_name_2])#add_node(Graph.nodes[edge_name_2])#edge_name_2)
                            #print(Graph.nodes[edge_name_2])
                            FlagEdge_2=True  

                        #if(FlagEdge_1==True and FlagEdge_2==True):
                        #    print(Graph.edges[Nodes_highlight])
                        #    highlight_graph.add_edges_from(Graph.edges[Nodes_highlight])

                        #print(position)
                        #nx.draw(highlight_graph, pos=position, ax=self.axis,node_color=highlight_color, node_size=node_size,with_labels=with_labels)

                        pos = nx.circular_layout(highlight_graph)  
                        nx.draw(highlight_graph, pos=pos2, ax=self.axis,node_color=highlight_color, node_size=node_size,with_labels=with_labels)
                        """
                        
                        print("Selected conduit: "+str(highlight_name))
                        break
                    
        self.fig.canvas.draw()        
        plt.show()

#make graph
def MakeGraph(Node_Names,
              Nodes_coord_X,
              Nodes_coord_Y,
              Cond_Names,
              Nodes_in,
              Nodes_out,
              Length,
              Roughness,
              Conduits_shape,
              Conduits_cross_sect_geom_1,
              Conduits_cross_sect_geom_2,
              Conduits_cross_sect_geom_3,
              Conduits_cross_sect_geom_4,
              Junc_Names,
              Junc_elevation,
              Junc_max_depth,
              Junc_init_depth,
              Storage_name,
              Storage_elevation,
              Storage_max_depth,
              Pump_name,
              Pump_From_Node,
              Pump_To_Node,
              Pump_Curve,
              Pump_Status,
              Pump_Sartup,
              Pump_Shutoff,
              Outfalls_name,
              Outfalls_elevation,
              Outfalls_type
             ):

    points=[]
    for g in range(0,len(Nodes_coord_X)):
        points.append((Nodes_coord_X[g],Nodes_coord_Y[g]))

    pos2={}
    for i in range(0,len(points)):
        pos2[Node_Names[i]] =  np.asarray((float(points[i][0]),float(points[i][1])))   
    
    Graph = nx.DiGraph(directed=True,strict=True)#Graph()
    #add nodes
    for l in range(0,len(Node_Names)):
        
        #print(float(points[l][0]))
        #print(float(points[l][1]))
        
        #check if this node is a junction (e.g. node with input output flows)
        isJunction=False
        indx=0
        junction_elevation=-1
        junction_max_depth=-1
        junction_init_depth=-1
        
        if(Node_Names[l] in Junc_Names):
            isJunction=True
            indx=Junc_Names.index(Node_Names[l] ) 
            junction_elevation=Junc_elevation[indx]
            junction_max_depth=Junc_max_depth[indx]
            junction_init_depth=Junc_init_depth[indx]

        isStrage=False
        storage_el=-1
        storage_max_d=-1
        indx=0
        if(Node_Names[l] in Storage_name):
            indx=Storage_name.index(Node_Names[l] )
            isStrage=True
            #isJunction=False #in case we switch off the flag that it is a junction a s storage can be junction as well, but we keep sxmth single
            storage_el=Storage_elevation[indx]
            storage_max_d=Storage_max_depth[indx]

        isOutfall=False
        Outf_elev=-1
        Outf_type="NOTFREE"
        if(Node_Names[l] in Outfalls_name):
            isOutfall=True
            indx=Outfalls_name.index(Node_Names[l])
            Outf_elev=Outfalls_elevation[indx]
            Outf_type=Outfalls_type[indx]
                
        Graph.add_node(Node_Names[l], 
                       pos=(float(points[l][0]), float(points[l][1])),
                       value=float(0.0), #actual water level                    
                       depth=0.92, #this defines the offset - e.g. the distance from the floor of the well to the out pipe (imposing delays of water flow)
                       saturation_falg=False,
                       sturation_level=0, #the water level cant go above this value, imitating flooding
                       water_loss_flag=True,#precipittaion, evaporation and other water loss things
                       water_loss_rate=0.87,
                       X=float(points[l][0]),
                       Y=float(points[l][1]),
                       #this indicates if this node is a junction
                       is_junction=isJunction,
                       junction_elevation=junction_elevation,
                       junction_max_depth=junction_max_depth,
                       junction_init_depth=junction_init_depth,
                       #if it is a torage
                       isStorage=isStrage,
                       storage_elevation=storage_el,
                       Storage_max_depth=storage_max_d,
                       #outfalls flag
                       isOutfall=isOutfall,
                       Outfall_elevation=Outf_elev,
                       Outfall_type=Outf_type
                      )
    
    graph_nodes_n=len(Node_Names)
    
    min_x_coord= np.min(np.asarray(points)[:,0])
    max_x_coord= np.max(np.asarray(points)[:,0])
    
    min_y_coord= np.min(np.asarray(points)[:,1])
    max_y_coord= np.max(np.asarray(points)[:,1])
    
    print("Map dimensions:")
    print("X min: " +str(min_x_coord))
    print("X max: " +str(max_x_coord))
    print("Y min: " +str(min_y_coord))
    print("Y max: " +str(max_y_coord))
    print("")
    print("Nodes number: "+str(graph_nodes_n))
    
    
    #add edges
    cnt_edge=0
    for i in range(0,len(Cond_Names)):
        indx_in=0
        indx_out=0
        for h in range(0,len(Node_Names)):
            if(Nodes_in[i]==Node_Names[h]):
                indx_in=h
            if(Nodes_out[i]==Node_Names[h]):
                indx_out=h
        #Graph.add_edge(indx_in, indx_out, egdes = cnt_edge)
        Graph.add_edge(Node_Names[indx_in], Node_Names[indx_out], egdes = cnt_edge, 
                       weight=0.82,
                       roughness=Roughness[i],
                       length=Length[i],
                       x_cross_section=Conduits_cross_sect_geom_1[i]
                      )
        cnt_edge=cnt_edge+1

    return Graph,points,pos2

#lorenz attractor
#https://matplotlib.org/stable/gallery/mplot3d/lorenz_attractor.html
class Lorenz:

    def __init__(self):
        
        self.dt=0.01
        self.num_steps=1000
        self.map=np.empty
        self.nRow = 50
        self.nCol = 50
        self.xyzs=np.empty
        self.xyz_hist=[]
        
    def lorenz(self,xyz, *, s=10, r=28, b=2.667):    
        
        """
        Parameters
        ----------
        xyz : array-like, shape (3,)
           Point of interest in three-dimensional space.
        s, r, b : float
           Parameters defining the Lorenz attractor.
        
        Returns
        -------
        xyz_dot : array, shape (3,)
           Values of the Lorenz attractor's partial derivatives at *xyz*.
        """
        
        x, y, z = xyz
        x_dot = s*(y - x)
        y_dot = r*x - y - x*z
        z_dot = x*y - b*z
        
        return np.array([x_dot, y_dot, z_dot])

    def Reset(self):
        self.xyzs=np.empty
        self.xyz_hist=[]
        self.map=np.empty

    def Lorenz_Run(self):
        
        #dt = 0.01         
        #num_steps = nRow*nCol
        
        if(self.num_steps!=self.nRow):
            self.num_steps=self.nRow
        if(len(np.shape(self.map)) == 0):
            self.map=np.zeros((self.nRow,self.nCol))
        sz=np.shape(self.map)
        
        if(sz[0]!=self.nRow or sz[1] != nCol):
            self.map=np.zeros((self.nRow,self.nCol))
            
        if(len(np.shape(self.xyzs)) == 0):
            self.xyzs=np.empty((self.num_steps + 1, 3))
            self.xyzs[0] = (0., 1., 1.05)  # Set initial values  
        else:
            sz=np.shape(self.xyzs)
            self.xyzs[0]=self.xyzs[sz[0]-1][:]
            
        #xyzs = np.empty((self.num_steps + 1, 3))  # Need one more for the initial values
        
        # Step through "time", calculating the partial derivatives at the current point
        # and using them to estimate the next point        
        for i in range(self.num_steps):
            self.xyzs[i + 1] = self.xyzs[i] + self.lorenz(self.xyzs[i]) * self.dt
            self.xyz_hist.append(np.asarray(self.xyzs[i + 1]))
            #fill the map
            for b in range(0,self.nCol):
                self.map[i][b]=self.xyzs[i + 1][0]

        return self.map.copy()
        
        # Plot
        #ax = plt.figure().add_subplot(projection='3d')
        
        #ax.plot(*xyzs.T, lw=0.5)
        #ax.set_xlabel("X Axis")
        #ax.set_ylabel("Y Axis")
        #ax.set_zlabel("Z Axis")
        #ax.set_title("Lorenz Attractor")
        #plt.show()

#this simulation is the running clouds over the graph (see above) spatial domain
#https://pypi.org/project/nlmpy/
from nlmpy import nlmpy
#https://geostat-framework.readthedocs.io/projects/gstools/en/v1.1.1/tutorial_01_srf.html

#generation of the terrain
def Terrain_Gen(method, nRow = 50, nCol = 50, h = 0.76, chaos_obj=None, const_feed_val=None,Time_series_from_file=[],time_series_offset=0):
    
    if(method=="nlmpy"):

        #print("ENTERING GENERATION ")
        arr=nlmpy.mpd(nRow=nRow, nCol=nCol, h=h)        
        #print("ARRAY SIZE "+str(np.shape(arr)))
        return arr,None
        
    if(method=="random_fields"):
        
        #https://geostat-framework.readthedocs.io/projects/gstools/en/v1.1.1/index.html
        from gstools import SRF, Gaussian
        model = Gaussian(dim=2, var=1, len_scale=10)
        x = y = np.linspace(0, nRow, nCol)#range(nRow)
        srf = SRF(model)
        #srf((x, y), mesh_type='structured')
        #srf.plot()
        return srf,None
        
    if(method=="trunc_power_law"):
        
        #https://geostat-framework.readthedocs.io/projects/gstools/en/v1.1.1/index.html
        from gstools import SRF, TPLStable
        x = y = np.linspace(0, 100, 100)
        model = TPLStable(
            dim=2,           # spatial dimension
            var=1,           # variance (C calculated internally, so that `var` is 1)
            len_low=0,       # lower truncation of the power law
            len_scale=10,    # length scale (a.k.a. range), len_up = len_low + len_scale
            nugget=0.1,      # nugget
            anis=0.5,        # anisotropy between main direction and transversal ones
            angles=np.pi/4,  # rotation angles
            alpha=1.5,       # shape parameter from the stable model
            hurst=0.7,       # hurst coefficient from the power law
        )
        srf = SRF(model, mean=1, mode_no=1000, seed=19970221, verbose=True)
        return srf,None
    if (method=="lorenz_attractor"):
        if(chaos_obj==None):
            chaos_obj=Lorenz()
            chaos_obj.nRow=nRow
            chaos_obj.nCol=nCol
            chaos_obj.num_steps=chaos_obj.nRow
        arr=chaos_obj.Lorenz_Run()
        return arr, chaos_obj    
    if(method=="const"):
        #print("Constant cloud generator")
        if(const_feed_val==None):
            const_val=0.3754
        else:
            const_val=const_feed_val
        arr=np.zeros([nRow, nCol], dtype = float) #np.zeros(nRow,nCol)#nlmpy.mpd(nRow=nRow, nCol=nCol, h=const_val)
        arr=arr+const_val
        
        return arr, None

    if(method=="from_file"):
        #nRow is the shift on the time series
        real_offset=0
        line=np.zeros(nRow)
        if(time_series_offset>len(Time_series_from_file)):
            pass        
        if((time_series_offset+nRow) > len(Time_series_from_file)) and (time_series_offset < len(Time_series_from_file)):
            real_offset=(time_series_offset+nRow) - len(Time_series_from_file)
            for i in range(0,real_offset): #np.shape(line)[0]):
                line[i] = Time_series_from_file[time_series_offset+i]
            for i in range(real_offset,nRow):
                line[i]=0
        if((time_series_offset+nRow) <= len(Time_series_from_file)):
            for i in range(0,nRow):
                line[i] = Time_series_from_file[time_series_offset+i]
                    
        arr = np.zeros([nRow, nCol], dtype = float)

        for i in range(0,nCol):
            arr[:,i] = line

        return arr, None
                                                        
def Coord_Tests(ax,bx,ay,by,min_x_coord,max_x_coord,min_y_coord,max_y_coord):
    print("___________")
    print("COORDINATES_EXTR.:")
    print("x max: "+str(max_x_coord))
    print("x min: "+str(min_x_coord))
    print("y max: "+str(max_y_coord))
    print("y min: "+str(min_y_coord))
    print("COEFS.:")
    print("a for x: "+str(ax))
    print("b for x: "+str(bx))

    print("a for y: "+str(ay))
    print("b for y: "+str(by))

    print("TEST:")
    print("x max -> nRow: "+str((max_x_coord-bx)/ax))
    print("x min -> 0: "+str((min_x_coord-bx)/ax))
    print("y max -> nCol: "+str((max_y_coord-by)/ay))
    print("y min -> 0: "+str((min_y_coord-by)/ay))

    print("_____________")
    print("_____________")
    print("")

def StickImages(step=2, map_1=None, map_2=None):
    
    if len(map_1) == 0:
        return None
    if len(map_2) == 0:
        return None
     
    map_shape1=np.shape(map_1)    
    map_shape2=np.shape(map_2)    

    #print("step: "+str(step))
    #print(map_shape1)
    #print(map_shape2)
    
    result_map = np.zeros(map_shape1)
    
    if(step>map_shape1[0]):
        result_map[:][:] = map_2[map_shape2[0]-step:map_shape2[0]-step+map_shape1[0]][:]
    else:
        result_map[0:step][:]=map_2[map_shape2[0]-step:map_shape2[0]][:]
        result_map[step:][:]=map_1[0:map_shape1[0]-step][:]
    
    return result_map

def CorrectWithTransitionMap(size, map_1=None,map_2=None,noise_lev=0.2):
    
    if len(map_1) == 0:
        return None
    if len(map_2) == 0:
        return None

    if(size==0):
        return map_2

    shp1=np.shape(map_1)
    shp2=np.shape(map_2)
    
    line1=map_1[0][:]
    line2=map_2[shp2[0]-1][:]
    trabsient_pattern=np.zeros((size,shp1[1]))

    max_val=0
    min_val=100000000000000
    for j in range(0,size):
        val1 = line1[j]
        val2 = line2[j]
        if(val1>max_val):
            max_val=val1
        if(val2>max_val):
            max_val=val2
        if(val1<min_val):
            min_val=val1
        if(val2<min_val):
            min_val=val2

    noise_ratio=(max_val-min_val)*noise_lev#this is partially not working
    
    for k in range(0,shp1[1]):
        
        line1_val=line1[k]
        line2_val=line2[k]
        dx=(line2_val-line1_val)/size
        #print("value 1: "+str(line1_val)+" , value 2: "+str(line2_val) + " , dx "+str(dx))       
        start_val=line2_val

        span = line2_val - line1_val
        
        for j in range(0,size):
            start_val=start_val-dx
            noisy_value=random.uniform(start_val - noise_ratio*span,start_val + noise_ratio*span)#(start_val-noise_ratio, start_val+noise_ratio)
            trabsient_pattern[j][k]=noisy_value#start_val
            #print(start_val)      
        #print("___________________")      
    
    updated_map_2=np.zeros((shp2[0]+size,shp2[1]))
    
    #print("new map: "+str(np.shape(updated_map_2)))
    #print("map 1: "  +str(np.shape(map_1)))
    #print("map 2:"   +str(np.shape(map_2)))
    
    updated_map_2[0:shp2[0]][:]=map_2[:][:]    
    updated_map_2[shp2[0]:size+shp2[0]][:]=trabsient_pattern[:][:]
    
    return updated_map_2    

# function to get unique values from list
def unique(list1):

    """
    # initialize a null list
    unique_list = []

    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if(len(unique_list)==0):
            unique_list.append(x)
        else:
            if x in unique_list:
                pass
            else:
                unique_list.append(x)
    """
    
    used = set()
    #mylist = [u'nowplaying', u'PBS', u'PBS', u'nowplaying', u'job', u'debate', u'thenandnow']
    unique = [x for x in list1 if x not in used and (used.add(x) or True)]
    
    return unique
    # print list
    #for x in unique_list:
    #    print x,    

class Watershed():

   MASK = -2
   WSHD = 0
   INIT = -1
   INQE = -3

   def __init__(self, levels = 256):
      self.levels = levels

   # Neighbour (coordinates of) pixels, including the given pixel.
   def _get_neighbors(self, height, width, pixel):
      return np.mgrid[
         max(0, pixel[0] - 1):min(height, pixel[0] + 2),
         max(0, pixel[1] - 1):min(width, pixel[1] + 2)
      ].reshape(2, -1).T

   def apply(self, image):
      current_label = 0
      flag = False
      fifo = deque()

      height, width = image.shape
      total = height * width
      labels = np.full((height, width), self.INIT, np.int32)

      reshaped_image = image.reshape(total)
      # [y, x] pairs of pixel coordinates of the flattened image.
      pixels = np.mgrid[0:height, 0:width].reshape(2, -1).T
      # Coordinates of neighbour pixels for each pixel.
      neighbours=[]
      for p in pixels:
          cur_val=self._get_neighbors(height, width, p)
          neighbours.append(cur_val)

      max_length=0
      for p in neighbours:
          if(len(p)>max_length):
              max_length=len(p)

      aligned_neighbours=[]
      for p in neighbours:
          pm=[]
          for kl in range(0,len(p)):
              pm.append(p[kl])
          if(len(p)<max_length):
              for j in range(0,max_length-len(p)):
                  zu=np.array([-1,-1])
                  pm.append(zu)
          aligned_neighbours.append(pm)
      neighbours=np.asarray(aligned_neighbours)
      #neighbours = np.array([self._get_neighbors(height, width, p) for p in pixels])
      
      
      if len(neighbours.shape) == 3:
         # Case where all pixels have the same number of neighbours.
         neighbours = neighbours.reshape(height, width, -1, 2)
      else:
         # Case where pixels may have a different number of pixels.
         neighbours = neighbours.reshape(height, width)
      

      indices = np.argsort(reshaped_image)
      sorted_image = reshaped_image[indices]
      sorted_pixels = pixels[indices]

      # self.levels evenly spaced steps from minimum to maximum.
      levels = np.linspace(sorted_image[0], sorted_image[-1], self.levels)
      level_indices = []
      current_level = 0

      # Get the indices that deleimit pixels with different values.
      for i in range(0,total):
         if sorted_image[i] > levels[current_level]:
            # Skip levels until the next highest one is reached.
            while sorted_image[i] > levels[current_level]: current_level += 1
            level_indices.append(i)
      level_indices.append(total)

      start_index = 0
      for stop_index in level_indices:
         # Mask all pixels at the current level.
         for p in sorted_pixels[start_index:stop_index]:
            labels[p[0], p[1]] = self.MASK
            # Initialize queue with neighbours of existing basins at the current level.
            for q in neighbours[p[0], p[1]]:
                if(q[0]!=-1 and q[1]!=-1):
                    # p == q is ignored here because labels[p] < WSHD
                    if labels[q[0], q[1]] >= self.WSHD:
                        labels[p[0], p[1]] = self.INQE
                        fifo.append(p)
                        break

         # Extend basins.
         while fifo:
            p = fifo.popleft()
            # Label p by inspecting neighbours.
            for q in neighbours[p[0], p[1]]:
               # Don't set lab_p in the outer loop because it may change.
               lab_p = labels[p[0], p[1]]
               lab_q = labels[q[0], q[1]]
               if lab_q > 0:
                  if lab_p == self.INQE or (lab_p == self.WSHD and flag):
                     labels[p[0], p[1]] = lab_q
                  elif lab_p > 0 and lab_p != lab_q:
                     labels[p[0], p[1]] = self.WSHD
                     flag = False
               elif lab_q == self.WSHD:
                  if lab_p == self.INQE:
                     labels[p[0], p[1]] = self.WSHD
                     flag = True
               elif lab_q == self.MASK:
                  labels[q[0], q[1]] = self.INQE
                  fifo.append(q)

         # Detect and process new minima at the current level.
         for p in sorted_pixels[start_index:stop_index]:
            # p is inside a new minimum. Create a new label.
            if labels[p[0], p[1]] == self.MASK:
               current_label += 1
               fifo.append(p)
               labels[p[0], p[1]] = current_label
               while fifo:
                  q = fifo.popleft()
                  for r in neighbours[q[0], q[1]]:
                     if labels[r[0], r[1]] == self.MASK:
                        fifo.append(r)
                        labels[r[0], r[1]] = current_label

         start_index = stop_index

      sz_labels=np.shape(labels)
      return labels

def WatershedsSeparation(height_map,levels=4,save_water_sheds=False,save_terrain_mapdata_folder="",close_figs_after=False):
       
    sz=np.shape(height_map)    
    c = (255*(height_map - np.min(height_map))/np.ptp(height_map)).astype(np.uint8)    
    im = PIL.Image.fromarray(c)    
    #watershed
    w=Watershed()
    labels = w.apply(height_map)    
    w2=Watershed()
    w2.levels=levels
    labels2 = w2.apply(height_map)
    
    return labels,labels2

In [8]:
#SIMULATION - SimulationS
def SimulationS(sim_counts=600,
                               Graph=None,
                               #terrain
                               terrain_sz_X=50,
                               terrain_sz_Y=50,
                               terrain_max_h=0.95,
                               #plot options
                               PLOT_CLOUDS=False,
                               GRADIENT_TRANSITION_SIZE=5,
                               CLOUDS_GENERATOR_TYPE="lorenz_attractor",
                               #node capacity adjustment
                               DROP_ALL_ACCUMULATED_WATER=True,
                               RUN_COORD_TEST=False,
                               NODES_RUN_OFF_WITH_TERRAIN_PROFILE=False,
                               Time_series_from_file=[]
               ):

    #checks
    if(Graph==None):
        return
    
    #user settings
    sim_num=sim_counts
    
    #clouds and terrain size
    nRow=terrain_sz_X
    nCol=terrain_sz_Y
    h=terrain_max_h
    
    #plot options
    PLOT_CLOUDS = PLOT_CLOUDS
    GRADIENT_TRANSITION_SIZE = GRADIENT_TRANSITION_SIZE
    CLOUDS_GENERATOR_TYPE = "const" #options - "nlmpy", "lorenz_attractor", "white_noise","const" - have to be added
    
    #turn on different simulation options
    DROP_ALL_ACCUMULATED_WATER=DROP_ALL_ACCUMULATED_WATER
    RUN_COORD_TEST = False
    NODES_RUN_OFF_WITH_TERRAIN_PROFILE = False
    
    #some objects to generate chaotic data
    chaos_attr=None
    
    #here we store the dynamics of the water versus time
    time_series=[] #here we store the 
    total_series=[]#total amout of water moved from the system
    
    input_rain_water_per_node=[] #what was the input for each node from rain
    input_tube_water_per_node=[] #what was the input of the water from meighbours (e.g. children nodes)
    
    target_nodes_list=[]
    for edge in nx.edges(Graph):
        target_nod=edge[1]    
        target_nodes_list.append(target_nod)    
    #remove duplicates
    s = []
    for node in target_nodes_list:
       if node not in s:
           s.append(node)
           time_series.append([])
           input_rain_water_per_node.append(0)
           input_tube_water_per_node.append(0)
           
    target_nodes_list=s
    
    #compute coordinates correspondencies
    ax=(max_x_coord-min_x_coord)/nRow
    bx=min_x_coord
    ay=(max_y_coord-min_y_coord)/nCol
    by=min_y_coord
    
    if(RUN_COORD_TEST==True):
        Coord_Tests(ax,bx,ay,by,min_x_coord,max_x_coord,min_y_coord,max_y_coord)
    
    #this are the clouds
    mat_show=None
    t_s_offset=0
    
    front_buf,chaos_attr = Terrain_Gen(CLOUDS_GENERATOR_TYPE, nRow=nRow, nCol=nCol, h=h, chaos_obj = chaos_attr, const_feed_val = h,Time_series_from_file=Time_series_from_file,time_series_offset=t_s_offset) #nlmpy.mpd(nRow=nRow, nCol=nCol, h=h)
    back_buf,chaos_attr  = Terrain_Gen(CLOUDS_GENERATOR_TYPE, nRow=nRow, nCol=nCol, h=h, chaos_obj = chaos_attr, const_feed_val = h,Time_series_from_file=Time_series_from_file,time_series_offset=t_s_offset) #nlmpy.mpd(nRow=nRow, nCol=nCol, h=h)
    back_buf=CorrectWithTransitionMap(GRADIENT_TRANSITION_SIZE, map_1 = front_buf, map_2 = back_buf)
    
    #plt.imshow(front_buf)
    #plt.imshow(back_buf)
    
    #predpare the terrain map
    print("_____________")
    print("__TERRAIN MAP__")
    
    terrain_map=nlmpy.mpd(nRow=nRow, nCol=nCol, h=h)
    l1,l2=WatershedsSeparation(terrain_map,levels=4)
    #check the sized of each cluster
    terrain_map_watersheds=l1
    terrain_wsh_labels = np.unique(terrain_map_watersheds) #unique(terrain_map_watersheds)
    terrain_wsh_size=[]
    for i in range(0,len(terrain_wsh_labels)):
        value=np.count_nonzero(terrain_map_watersheds == terrain_wsh_labels[i])#terrain_map_watersheds.count(terrain_wsh_labels[i])
        terrain_wsh_size.append(value)
        
    #print(terrain_wsh_size)
    #print("------------------")
    #print(terrain_wsh_labels)
    #-----------------------------
    
    cnt=0
    sim_step=0
    
    for i in range(0,sim_num):
            
        if(cnt > nRow + GRADIENT_TRANSITION_SIZE):
            t_s_offset = t_s_offset + nRow
            sz_front_buf = np.shape(front_buf)
            front_buf = back_buf[0:sz_front_buf[0]][:]
            back_buf,chaos_attr = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr, const_feed_val=h,
                                              Time_series_from_file=Time_series_from_file,
                                              time_series_offset=t_s_offset) 
            back_buf=CorrectWithTransitionMap(GRADIENT_TRANSITION_SIZE, map_1 = front_buf, map_2 = back_buf)
            cnt=0
    
        mat_show=StickImages(step=cnt, map_1 = front_buf, map_2 = back_buf)
        #mat_show[:][0:cnt]=back_buf[:][0:cnt]
        cnt=cnt+1
            
        #simulate the input into the graph node
        #find the coordinate that corresponds to the coordinates on the *.inp graph
        for j in range(0,graph_nodes_n):
                    
            pos_x=points[j][0]
            pos_y=points[j][1]      
    
            map_x=int((pos_x-bx)/ax)
            map_y=int((pos_y-by)/ay)
    
            if(map_x>=nRow):
                map_x=nRow-1
            if(map_y>=nCol):
                map_y=nCol-1       
    
            #upate values 
            value=mat_show[map_x,map_y]
            #nx.set_node_attributes(Graph, value, name="value")
            #fill in the nodes with water
            if(NODES_RUN_OFF_WITH_TERRAIN_PROFILE==False):
                #if the nodes are 
                Graph.nodes[Node_Names[j]]["value"]=Graph.nodes[Node_Names[j]]["value"] + value
            else:
                #chekc the label of the terrain first
                label = terrain_map[map_x][map_y]
                total_water_runoff=0
                for a in range(0,nRow):
                    for b in range(0,nCol):
                        if(terrain_map_watersheds[a][b]==label):
                            total_water_runoff = total_water_runoff + mat_show[a,b]
                Graph.nodes[Node_Names[j]]["value"]=Graph.nodes[Node_Names[j]]["value"] + total_water_runoff
                #this is simplified version with averaging
                #indx = terrain_wsh_labels.index(label)
                #watershed_size = terrain_wsh_size[indx]            
                
            #gh=nx.get_node_attributes(Graph, name="value")
            #print("Map value")
            #print(value)
            #print("installed value")
            #print(gh)
            
        #make inference and run simulations
        
        total=0 #total water change    
        
        for edge in nx.edges(Graph):
            
            #this is water propagation modelling
            weight=Graph.get_edge_data(*edge)["weight"]
            out_n    = edge[0]     
            target_n = edge[1]
            out_n_val=Graph.nodes[out_n]["value"]
            target_n_val=Graph.nodes[target_n]["value"]  
    
            value_out_n    = out_n_val
            value_target_n = target_n_val
            
            #we check if the depth level has to be taken into account (this imposes delay)
            if(Graph.nodes[out_n]["depth"]!=0):
                if(value_out_n>=Graph.nodes[out_n]["depth"]):
                    value_out_n=np.max(value_out_n-Graph.nodes[out_n]["depth"],0)
                else:
                    value_out_n=0 
                    
            #water propagation functions
            tmp  = value_out_n*weight+value_target_n # Do something useful with the edges      
            tmp1 = tmp
            
            #check if water level is too high in the target node and if it is saturated
            if(Graph.nodes[target_n]["saturation_falg"]==True):
                if (tmp1>=Graph.nodes[target_n]["sturation_level"]):
                    tmp1=Graph.nodes[target_n]["sturation_level"]
                        
            #fill time series
            #indx=target_nodes_list.index(target_n)
            #total water change     
            #time_series[indx].append(tmp1)
            total=total+tmp
    
        #update the time series
        for node in target_nodes_list:
            
            tmp1=Graph.nodes[node]["value"]
            indx=target_nodes_list.index(node)
            time_series[indx].append(tmp1)
            
        #update the water level in all nodes
        if(DROP_ALL_ACCUMULATED_WATER!=False):
            for j in range(0,graph_nodes_n):
                cur_value=Graph.nodes[Node_Names[j]]["value"]
                if(Graph.nodes[out_n]["depth"]==0):        
                    Graph.nodes[Node_Names[j]]["value"]=0
                else:
                    if(cur_value<Graph.nodes[Node_Names[j]]["depth"]):
                        Graph.nodes[Node_Names[j]]["value"]=0
                    else:
                        Graph.nodes[Node_Names[j]]["value"]=Graph.nodes[Node_Names[j]]["value"]
                        
        # update values in all target nodes with respect to water losses (if those imposed)
        for j in range(0,graph_nodes_n):        
            #add water losses here (becasue of precipitation, for example)
            if (Graph.nodes[Node_Names[j]]["water_loss_flag"]==True):
                water_loss_rate=Graph.nodes[Node_Names[j]]["water_loss_rate"]
                value=Graph.nodes[Node_Names[j]]["value"]
                value=water_loss_rate*value
                Graph.nodes[Node_Names[j]]["value"]=value                                            
                    
        #print("DATA for sim.num. "+str(sim_step)+": "+str(total))
        total_series.append(total)
        sim_step=sim_step + 1         
            
        #in case we sould like to check the clouds distributon and how it evolves
        if PLOT_CLOUDS==True:
    
            if(sim_step==0):
                print("_____________")
                print("__CLOUDS MAP__")
    
            
            print("Simulation step: "+str(sim_step))
            plt.matshow(mat_show)
            plt.colorbar()
            plt.show()
        
    print("_____________")
    print("__NODES MAP__")
    
    blank_image = np.zeros((nRow,nCol,3), np.uint8)
    
    for j in range(0,graph_nodes_n):
        pos_x=points[j][0]
        pos_y=points[j][1]
        map_x=int((pos_x-bx)/ax)
        map_y=int((pos_y-by)/ay)
        if(map_x>=nRow):
            map_x=nRow-1
        if(map_y>=nCol):
            map_y=nCol-1
        blank_image[map_x,map_y,0]=252
    
    plt.imshow(blank_image)
    plt.show()
    
    print("_____________")
    print("__SIMULATION COMPLETE__")

#pywr simulation
#duration_type = sec, min, hours, days
def PywrSimulation(network=None,sim_start=None, sim_dur=500,duration_type="sec"):
    #https://github.com/pywr/pywr?tab=readme-ov-file
    #https://www.sciencedirect.com/science/article/pii/S1364815219307133 - paper
    
    from pywr.core import Model, Input, Output
    from pywr.core import Timestepper
    from pywr.recorders import NumpyArrayNodeRecorder

    if(sim_start is None):
        #https://www.programiz.com/python-programming/datetime/current-datetime
        now = datetime.now()
        sim_start = now.strftime("%d-%m-%Y-%H-%M-%S")
        
    sim_delta=None    
    sim_start_days=now.strftime("%Y-%m-%d")
    if(duration_type=="sec"):
       b = now + timedelta(seconds=sim_dur)
       sim_delta=timedelta(seconds=1)
    if(duration_type=="min"):
       b = now + timedelta(minutes=sim_dur)
       sim_delta=timedelta(minutes=1)
    if(duration_type=="hours"):
       b = now + timedelta(hours=sim_dur)
       sim_delta=timedelta(hours=1)
    sim_end=b.strftime("%d-%m-%Y %H:%M:%S")
    sim_end_days=b.strftime("%Y-%m-%d")
        
    print("")
    print("***********************")
    print("****Simulation Pywr****")
    print("***********************")
    print("Sim. parems:")
    print("Sim. start: "+str(sim_start))
    print("Sim duration: "+ str(sim_dur)+str(" ")+duration_type)
    print("Sim.end: "+str(sim_end))
    print("")
    print("Sim. start day:"+sim_start_days)
    print("Sim. end day:"+sim_end_days)

    #start simulation
    # create a model (empty network)
    model = Model()
    #fill with inputs outputs
    supply = Input(model, name='supply')
    demand = Output(model, name='demand')
    # create a connection from the supply to the demand
    supply.connect(demand)
    model.timestepper = Timestepper(
            pandas.to_datetime(sim_start_days),  # first day
            pandas.to_datetime(sim_end_days),  # last day
            sim_delta#timedelta(1)  # interval
            )
    recorder = NumpyArrayNodeRecorder(model, supply)
    # lets get this party started!
    model.run()

def EPANET(inp_model="",network=None,sim_start=None, sim_dur=500,duration_type="sec"):
        
    #https://github.com/OpenWaterAnalytics/EPyT
        
    if(sim_start is None):
        #https://www.programiz.com/python-programming/datetime/current-datetime
        now = datetime.now()
        sim_start = now.strftime("%d-%m-%Y-%H-%M-%S")
        
    sim_delta=None    
    sim_start_days=now.strftime("%Y-%m-%d")
    if(duration_type=="sec"):
       b = now + timedelta(seconds=sim_dur)
       sim_delta=timedelta(seconds=1)
    if(duration_type=="min"):
       b = now + timedelta(minutes=sim_dur)
       sim_delta=timedelta(minutes=1)
    if(duration_type=="hours"):
       b = now + timedelta(hours=sim_dur)
       sim_delta=timedelta(hours=1)
    sim_end=b.strftime("%d-%m-%Y %H:%M:%S")
    sim_end_days=b.strftime("%Y-%m-%d")

    if(inp_model==""):
        print("cant open inp model")
        return
    else:
        try:
            d = epanet('Net1.inp')
            d.getNodeCount()
            d.getNodeElevations()
        except Exception as ex:
            print("cant open given file: "+inp_model)
            return
        
    print("")
    print("***********************")
    print("****Simulation Pywr****")
    print("***********************")
    print("Model: "+inp_model)
    print("Sim. params:")
    print("Sim. start: "+str(sim_start))
    print("Sim duration: "+ str(sim_dur)+str(" ")+duration_type)
    print("Sim.end: "+str(sim_end))
    print("")
    print("Sim. start day:"+sim_start_days)
    print("Sim. end day:"+sim_end_days)

#------------------------------
# WNTR - the simulations 
#------------------------------

#fornetwork elements check this instruction
#https://reganmurray-wntr.readthedocs.io/en/stable/waternetworkmodel.html

def WNTR(graph=None,sim_start=None, sim_dur=500,duration_type="sec",
                                                useWaterInputs=False, #use this option to introduce the water injections into the system
                                                NodeNames=[],
                                                PatternsTimeSerries=[]
                                                ):
    #https://wntrdev.readthedocs.io/en/latest/waternetworkmodel.html#build-a-model-from-an-inp-file
    import wntr
    G=graph
    
    if(sim_start is None):
        #https://www.programiz.com/python-programming/datetime/current-datetime
        now = datetime.now()
        sim_start = now.strftime("%d-%m-%Y-%H-%M-%S")

    sim_duration_in_seconds=sim_dur
    sim_delta=None    
    sim_start_days=now.strftime("%Y-%m-%d")
    if(duration_type=="sec"):
       b = now + timedelta(seconds=sim_dur)
       sim_delta=timedelta(seconds=1)
    if(duration_type=="min"):
       b = now + timedelta(minutes=sim_dur)
       sim_delta=timedelta(minutes=1)
       sim_duration_in_seconds=sim_duration_in_seconds*60
    if(duration_type=="hours"):
       b = now + timedelta(hours=sim_dur)
       sim_delta=timedelta(hours=1)
       sim_duration_in_seconds=sim_duration_in_seconds*60*60
    sim_end=b.strftime("%d-%m-%Y %H:%M:%S")
    sim_end_days=b.strftime("%Y-%m-%d")

    
    if(graph is None):        
        print("Graph is not passed")
        return        
    else:
        wn,n_names,l_names=WNTR_show_elevation_for_model(G=graph,
                                                         useWaterInputs=useWaterInputs, 
                                                         NodeNames=NodeNames, 
                                                         PatternsTimeSerries=PatternsTimeSerries)
       
           
    print("")
    print("***********************")
    print("****Simulation WNTR****")
    print("***********************")    
    print("Sim. params:")
    print("Sim. start: "+str(sim_start))
    print("Sim duration: "+ str(sim_dur)+str(" ")+duration_type)
    print("Sim.end: "+str(sim_end))
    print("")
    print("Sim. start day:"+sim_start_days)
    print("Sim. end day:"+sim_end_days)
    print("")
    
    #https://wntrdev.readthedocs.io/en/latest/waternetworkmodel.html#build-a-model-from-an-inp-file     
    #set simulation options
    wn.options.time.duration+=sim_duration_in_seconds #should be give in seconds    
    wn.options.time.hydraulic_timestep  =1
    wn.options.time.quality_timestep    =1
    wn.options.time.rule_timestep       =1
    wn.options.time.pattern_timestep    =1
    wn.options.time.report_timestep   =1
    
    sim = wntr.sim.EpanetSimulator(wn)
    results = sim.run_sim()
    
    print("")
    print("Simulations are complete")
    print("Accessible data:")
    
    #https://wntrdev.readthedocs.io/en/latest/resultsobject.html
    node_keys = results.node.keys()
    link_keys = results.link.keys()
    print("for nodes - ")
    print(node_keys)
    print("for pipes - ")
    print(link_keys)
    print("")
    """
    n_names_updated=[]
    result_for_nodes=[]
    cnt=0
    for v in node_keys:
        result1 = results.node[v]   
        cnt_1=0
        print(result1)
        for m in n_names:
            result_node=result1[:,m]
            if(result_node==None)==False:
                if(cnt==0):
                    sub_list=[]
                    sub_list.append(result_node)
                    result_for_nodes.append(sub_list)
                    n_names_updated.append(m)
                else:
                    result_for_nodes[cnt_1].append(list(result_node))
                cnt_1=cnt_1+1
        cnt=cnt+1               

    p_names_updated=[]
    result_for_pipes=[]
    cnt=0
    for v in link_keys:
        result1 = results.link[v]   
        cnt_1=0
        for m in l_names:
            result_link=result1[:,m]
            if(result_link==None)==False:
                if(cnt==0):
                    sub_list=[]
                    sub_list.append(result_link)
                    result_for_pipes.append(sub_list)
                    p_names_updated.append(m)
                else:
                    result_for_pipes[cnt_1].append(list(result_link))
                    
                cnt_1=cnt_1+1
        cnt=cnt+1               
    """
    return wn,results,n_names,l_names

def WNTR_show_elevation_for_model(G=None,show_elevation=False,show_population=False,
                                  useWaterInputs=False, 
                                  NodeNames=[], 
                                  PatternsTimeSerries=[],
                                  #this is the water quality
                                  Pollution_introduction_flag=False,
                                  Pollution_nodes_list=[],
                                  Pollution_time_series=[],
                                  Pollution_pattern_params=[]
                                 ):
    
    #also check - https://usepa.github.io/WNTR/examples.html
    import wntr
    wn = wntr.network.WaterNetworkModel()
    Used_Nodes_Names=[]
    Pipes_Names=[]
    for edge in  G.edges(data=True):#u,outer_d in G.nodes(data=True):
        
            n1=edge[0]
            n2=edge[1]        

            """
            print("")
            print("--------------")
            print("Node names")
            print(n1)
            print(n2)
            print("")
            """
        
            # NODE 1        
            if((n1 in Used_Nodes_Names)==False):
                
                Used_Nodes_Names.append(n1)
                #if(len(Used_Nodes_Names)==0):
                #    Used_Nodes_Names.append(n1)    
                #print(n1+" junction flag: "+str((G.nodes[n1]["is_junction"])))
                #print(n1+" storage flag: "+str((G.nodes[n1]["isStorage"])))
                #print(n1+" outfall flag: "+str((G.nodes[n1]["isStorage"])))    
                
                if(int(G.nodes[n1]["is_junction"])==True): #and G.nodes[n2]["is_junction"]==1):
                    #check if we would like to have the water input for this node
                    patern_name="pat"
                    if(useWaterInputs==True):
                        if(n1 in NodeNames)==True:
                            index=NodeNames.index(n1)
                            patern_name=str('pat_'+n1)
                            wn.add_pattern(patern_name, PatternsTimeSerries[index])
                    
                    wn.add_junction(n1,base_demand=0.01,
                                               demand_pattern=patern_name, 
                                               elevation=float(G.nodes[n1]["junction_elevation"]),
                                               coordinates=(float(G.nodes[n1]["X"]),float(G.nodes[n1]["Y"]))
                                           )
                 #   print("Node "+n1+" is added as junction")
                
                if(int(G.nodes[n1]["isStorage"])==True):
                    wn.add_reservoir(n1,base_head=float(G.nodes[n1]["Storage_max_depth"]),
                                        #elevation=float(G.nodes[n1]["storage_elevation"]),
                                        coordinates=(float(G.nodes[n1]["X"]),float(G.nodes[n1]["Y"]))
                                    )
                  #  print("Node "+n1+" is added as storage")
    
                if(G.nodes[n1]["isOutfall"]==True):
                    #we add it as a hige reservuar
                    wn.add_reservoir(n1,base_head=200,
                                        #elevation=float(G.nodes[n1]["Outfall_elevation"]),
                                        coordinates=(float(G.nodes[n1]["X"]),float(G.nodes[n1]["Y"]))
                                    )
                   # print("Node "+n1+" is added as outfall (storage)")
               
             
            #NODE 2
        
            if((n2 in Used_Nodes_Names)==False):
                
                Used_Nodes_Names.append(n2)
                #print(n2+" junstion flag: "+str((G.nodes[n2]["is_junction"])))
                #print(n2+" stoarge flag: "+str((G.nodes[n2]["isStorage"])))
                #print(n2+" outfall flag: "+str((G.nodes[n2]["isOutfall"])))
            
                if(G.nodes[n2]["is_junction"]==True):
                    
                    patern_name="pat"
                    if(useWaterInputs==True):
                        if(n2 in NodeNames)==True:
                            index=NodeNames.index(n2)
                            patern_name=str('pat_'+n2)
                            wn.add_pattern(patern_name, PatternsTimeSerries[index])
                    
                    wn.add_junction(n2,base_demand=0.01,
                                       demand_pattern=patern_name, 
                                       elevation=float(G.nodes[n2]["junction_elevation"]),
                                       coordinates=(float(G.nodes[n2]["X"]),float(G.nodes[n2]["Y"]))
                                   )
                 #   print("Node " + n2 + " is added as junction")
    
                
                if(G.nodes[n2]["isStorage"]==True):
                    wn.add_reservoir(n2,base_head=float(G.nodes[n2]["Storage_max_depth"]),
                                        #elevation=float(G.nodes[n2]["storage_elevation"]),
                                        coordinates=(float(G.nodes[n2]["X"]),float(G.nodes[n2]["Y"]))
                                    )
                  #  print("Node "+ n2 + " is added as stoarge")
    
                if(G.nodes[n2]["isOutfall"]==True):
                    #we add it as a hige reservuar
                    wn.add_reservoir(n2,base_head=200,
                                        #elevation=float(G.nodes[n2]["Outfall_elevation"]),
                                        coordinates=(float(G.nodes[n2]["X"]),float(G.nodes[n2]["Y"]))
                                    )
                   # print("Node "+n2+" is added as outfall (storage)")                                       
                                       
            pipe_name=str(n1)+"-"+str(n2)
        
            Pipes_Names.append(pipe_name)
            #pipe_name=str(G.get_edge_data(*edge)["egdes"])
            length=float(G.get_edge_data(*edge)["length"])
            diameter=float(G.get_edge_data(*edge)["x_cross_section"])#['weight'])
            roughness=float(G.get_edge_data(*edge)['roughness'])                
                
            wn.add_pipe(pipe_name, start_node_name=n1, 
                                   end_node_name=n2, 
                                   length=length,
                                   diameter=diameter, 
                                   roughness=roughness, 
                                   minor_loss=0)     
            #print ("Pipe between nodes "+n1+" and "+n2+" is added....")
            #print("--------------")        
            #get_edge_data(*edge)["weight"]
            #('101', '102a', {'egdes': 332, 'weight': 0.82, 'roughness': '0.011764706', 'length': '65.878'})    
   
    
    if(show_elevation==True):
        %matplotlib qt 
        f2 = plt.figure()    
        wntr.graphics.plot_network(wn, node_attribute='elevation')        
        plt.show()
        %matplotlib inline 
        print("Elevation model is shown")

    if(show_population==True):
        %matplotlib qt 
        f3 = plt.figure()    
        #wntr.graphics.plot_network(wn, node_attribute='elevation')        
        population = wntr.metrics.population(wn)
        node_range_show=G.number_of_nodes()
        wntr.graphics.plot_interactive_network(wn, node_attribute=population, node_range=[0,graphics],auto_open=False) 
        plt.show()
        %matplotlib inline         
        print("Population model is shown")

    #here we add pollution story
    #an example can be found in - https://usepa.github.io/WNTR/waterquality.html
    wn.options.quality.parameter = 'CHEMICAL'
    if(Pollution_introduction_flag==True):        
        for l in range(0,len(Pollution_nodes_list)):
            pollution_source_pattern = wntr.network.elements.Pattern.binary_pattern('PollutionSourcePattern',
                                                                          start_time=Pollution_pattern_params[l][0], 
                                                                          end_time=Pollution_pattern_params[l][1], 
                                                                          duration=Pollution_pattern_params[l][2],
                                                                          step_size=wn.options.time.pattern_timestep)
            node_name=Pollution_nodes_list[l]
            pattern_name=str('Pollution_SourcePattern_for_node_'+node_name)
            source_name=str("Source_for_node_" + node_name)
            wn.add_pattern(pattern_name, source_pattern)       
            wn.add_source(source_name, node_name, 'SETPOINT', 1000, pattern_name)
    
    return wn,Used_Nodes_Names,Pipes_Names

#____________________________________________________________________
#____________PANDAPIPES MODELLING____________________________________
#____________________________________________________________________
#____________________________________________________________________

def Pandapipes_makeNet(graph=None):
    #https://github.com/e2nIEE/pandapipes/blob/develop/tutorials/standard_libraries.ipynb
    if(graph==None):
        return None,None,None
    wn = pp.create_empty_network(fluid="water")
    Used_Nodes_Names=[]
    junction_list=[]
    Pipes_Names=[]
    for edge in  G.edges(data=True):#u,outer_d in G.nodes(data=True):
        
            n1=edge[0]
            n2=edge[1]
        
            # NODE 1        
            if((n1 in Used_Nodes_Names)==False):
                junction1 = pp.create_junction(wn, pn_bar=1.0, tfluid_k=293.15, name=n1)
                Used_Nodes_Names.append(n1)                  
                
                if(int(G.nodes[n1]["isStorage"])==True or G.nodes[n1]["isOutfall"]==True):
                    sink = pp.create_sink(net, junction=junction1, mdot_kg_per_s=0.045, name="Sink")    
               
                junction_list.append(junction1)               
             
            #NODE 2
        
            if((n2 in Used_Nodes_Names)==False):
                junction2 = pp.create_junction(wn, pn_bar=1.0, tfluid_k=293.15, name=n2)
                Used_Nodes_Names.append(n2)
                
                if(int(G.nodes[n1]["isStorage"])==True or G.nodes[n1]["isOutfall"]==True):
                    sink = pp.create_sink(net, junction=junction2, mdot_kg_per_s=0.045, name="Sink")         

                junction_list.append(junction2)         
                                       
            pipe_name=str(n1)+"-"+str(n2)
        
            Pipes_Names.append(pipe_name)
            #pipe_name=str(G.get_edge_data(*edge)["egdes"])
            length=float(G.get_edge_data(*edge)["length"])
            diameter=float(G.get_edge_data(*edge)["x_cross_section"])#['weight'])
            roughness=float(G.get_edge_data(*edge)['roughness'])                
                
            pipe = pp.create_pipe_from_parameters(net, from_junction=junction1, to_junction=junction2, length_km=length, diameter_m=diameter, name=pipe_name)

    return wn,Used_Nodes_Names,Pipes_Names


#__________________________________________________________________
#_____________EpyT generator_______________________________________

def EpyT_makeNet(graph=None, useWaterInputs=True,NodeNames=[],PatternsTimeSerries=[]):
    
    #https://github.com/KIOS-Research/EPyT/blob/main/epyt/examples/EX1_Plot_network_topology.ipynb    
    #https://pypi.org/project/epyt/
    G=graph    
    if(graph==None):
        return None, None, None

    BASE_DEMAND=0.0005
    
    #load model that will be earsed and new model will be introduced on top
    current_working_directory = os.getcwd()
    inp_init_path=current_working_directory+"\\EpyT_init_model.inp"
    d = epanet(inp_init_path)
    link_count = d.getLinkCount()
    nodes_count= d.getNodeCount()
    rules_ID = d.getRules()
    control_count=d.getControlRulesCount()
    print("Initialization of EpyT model:")
    print(" - number of nodes: "+str(nodes_count))
    print(" - number of links: "+str(link_count))
    print(" - number of rules: "+str(len(rules_ID)))
    print(" - number of controls: "+str(control_count))
    
    print("")
    nodes_n=d.getNodeNameID()
    links_n=[]
    for i in range(0,link_count):
        try:
            cur_name=d.getLinkNameID(i)
            links_n.append(cur_name)
        except:
            pass
    
    for m in links_n:
        d.deleteLink(m)
    for k in nodes_n:
        d.deleteNode(k)
    for v in range(0,control_count):
        d.deleteControls()
    d.deleteRules()
    
    print("Cleaned EpyT model:")
    print(" - number of nodes: "+str(d.getLinkCount()))
    print(" - number of links: "+str(d.getNodeCount()))
    print(" - number of rules: "+str(len(d.getRules())))
    print(" - number of controls: "+str(d.getControlRulesCount()))
    print("")
    print("Initialize networkX model...")
    print("")
        
    Used_Nodes_Names=[]
    Pipes_Names=[]
    
    for edge in  G.edges(data=True): #u,outer_d in G.nodes(data=True):      
        
        n1=edge[0]
        n2=edge[1]           
      
        if((n1 in Used_Nodes_Names)==False):                                             
            if(int(G.nodes[n1]["is_junction"])==True):     
                #https://epanet-python-toolkit-epyt.readthedocs.io/en/latest/api.html
                x_coord=float(G.nodes[n1]["X"])
                y_coord=float(G.nodes[n1]["Y"])
                coordinates=[x_coord,y_coord]
                junctionElevation = float(G.nodes[n1]["junction_elevation"])
                demand = BASE_DEMAND                          
                if (useWaterInputs==True) and (n1 in NodeNames)==True:
                    index=NodeNames.index(n1)
                    patern_name=str('pat_'+n1)
                    time_series=PatternsTimeSerries[index]                    
                    d.addPattern(patern_name,time_series)                         
                    #ptrn=wn.getPatternNameID(patern_name)                   
                    d.addNodeJunction(n1,coordinates,junctionElevation,demand,patern_name)             
                else:                    
                    d.addNodeJunction(n1,coordinates,junctionElevation,demand)   
                                
            if(int(G.nodes[n1]["isStorage"])==True or int(G.nodes[n1]["isOutfall"])==True):                 
                x_coord=float(G.nodes[n2]["X"])
                y_coord=float(G.nodes[n2]["Y"])
                reservoirCoords = [x_coord, y_coord]
                reservoirIndex = d.addNodeReservoir(n1, reservoirCoords)
                
            Used_Nodes_Names.append(n1)    
           
        # NODE 2     
       
        if((n2 in Used_Nodes_Names)==False):               
            if(int(G.nodes[n2]["is_junction"])==True):         

                #https://epanet-python-toolkit-epyt.readthedocs.io/en/latest/api.html
                x_coord=float(G.nodes[n2]["X"])
                y_coord=float(G.nodes[n2]["Y"])
                coordinates=[x_coord,y_coord]
                junctionElevation = float(G.nodes[n2]["junction_elevation"])
                demand = BASE_DEMAND           
                                
                if (useWaterInputs==True) and (n2 in NodeNames)==True:
                    index=NodeNames.index(n2)
                    patern_name=str('pat_'+n2)
                    time_series=PatternsTimeSerries[index]                    
                    d.addPattern(patern_name,time_series)                         
                    d.addNodeJunction(n2,coordinates,junctionElevation,demand,patern_name)              
                else:                    
                    d.addNodeJunction(n2,coordinates,junctionElevation,demand)   
                    
            if(int(G.nodes[n2]["isStorage"])==True or int(G.nodes[n2]["isOutfall"])==True):                
                x_coord=float(G.nodes[n2]["X"])
                y_coord=float(G.nodes[n2]["Y"])
                reservoirCoords = [x_coord, y_coord]
                reservoirIndex = d.addNodeReservoir(n2, reservoirCoords)   
                
            Used_Nodes_Names.append(n2)       
        #https://epanet-python-toolkit-epyt.readthedocs.io/en/latest/api.html        
        pipeID=str(n1)+"_"+str(n2)
        length=float(G.get_edge_data(*edge)["length"])
        diameter=float(G.get_edge_data(*edge)["x_cross_section"])#['weight'])
        roughness=float(G.get_edge_data(*edge)['roughness'])        
        minorLossCoeff=0.0       
        #link_names = wn.getLinkNameID()
        #print(pipeID)
        #if(True):
        #    print(length)
        #    print(diameter)
        #    print(roughness)
        #    print(minorLossCoeff)           
        pipe=d.addLinkPipe(pipeID, n2, n1,length,diameter,roughness,minorLossCoeff)  
        Pipes_Names.append(pipeID)
        #print("pipe added")
        
    print("") 
    print("Transfered from NetworkX EpyT model:")
    print(" - number of nodes: "+str(d.getLinkCount()))
    print(" - number of links: "+str(d.getNodeCount()))
    print(" - number of rules: "+str(len(d.getRules())))
    print(" - number of controls: "+str(d.getControlRulesCount()))
    print("")    
        
    return d, Used_Nodes_Names, Pipes_Names    
#____________________________________________________________________
#_____________________StormReactor___________________________________
#____________________________________________________________________
#https://github.com/kLabUM/StormReactor
def StormReactor_Model(graph=None,path=""):
    if(path!=""):
        pass
#____________________________________________________________________
#__________GENERATE SPATIAL DOMAIN___________________________________
#____________________________________________________________________



def GenerateSpatialDomain(scenario="Clouds",
                          nodes=None,
                          time_series_length=500,
                          nodes_X=None,
                          nodes_Y=None,
                          spatial_domain_nRows=50,
                          spatial_domain_nCol=50,                                             
                          rand_gen_type="nlmpy",#"const","nlmpy", "lorenz_attractor", "white_noise","const"
                          #clouds
                          clouds_max_h=0.76,
                          clouds_grad_transition=0,
                          #terrain max height
                          terrain_max_h=0.65,
                          #save everything related to the simulation
                          save_history=True,
                          histry_dir="",
                          #this are the subdir names
                          sub_dir_clouds_history="Clouds",
                          sub_dir_terrain_hist="Terrain",
                          sub_dir_time_series_nodes_intake="Nodes_Intake",
                          sub_dir_nodes_params="Nodes_params",
                          #this is in case the spatial domain is recorded from the file
                          Clouds_timeseries_from_file=[]
                          ):    
    print("ENTERING GenerateSpatialDomain")
     #variables
    nRow=spatial_domain_nRows
    nCol=spatial_domain_nCol
    h=clouds_max_h
    h_terrain=terrain_max_h
    
    #prepare the time series and return variables
    #modes inputs
    rain_node_input=[]
    for i in range(0,len(nodes)):
        rain_node_input.append([])
    #this is terrain
    terrain_map=None
    watershed_map=None
        
    #prepare the coordinates convertions
    min_x_coord= np.min(np.asarray(nodes_X))
    max_x_coord= np.max(np.asarray(nodes_X))
    min_y_coord= np.min(np.asarray(nodes_Y))
    max_y_coord= np.max(np.asarray(nodes_Y))
    ax=(max_x_coord-min_x_coord)/nRow
    bx=min_x_coord
    ay=(max_y_coord-min_y_coord)/nCol
    by=min_y_coord

    #service data
    map_watersheds_accumulation_water=[]

    print("GenerateSpatialDomain - initialization passed")
    
    #check folders
    if(save_history==True):
         if not os.path.exists(histry_dir):
                os.makedirs(histry_dir)
         clouds_data=histry_dir+"\\"+sub_dir_clouds_history
         if not os.path.exists(clouds_data):
                os.makedirs(clouds_data)
         time_series_intake=histry_dir+"\\"+sub_dir_time_series_nodes_intake
         if not os.path.exists(time_series_intake):
             os.makedirs(time_series_intake)
         terrain_data=histry_dir+"\\"+sub_dir_terrain_hist
         if not os.path.exists(terrain_data):
             os.makedirs(terrain_data)
         terrain_water_accum=terrain_data+"\\Water_Accum_Per_WaterSheds"
         if not os.path.exists(terrain_water_accum):
             os.makedirs(terrain_water_accum)
         nodes_params=histry_dir+"\\"+sub_dir_nodes_params
         if not os.path.exists(nodes_params):
             os.makedirs(nodes_params)

         print("GenerateSpatialDomain - folders for saving data are created")

    #simple coud, the water falls from cloud directly into the node without watersheds or any other interfaces inbwteen
    #pointwise water introduction
    print("Entering scenario - " + str(scenario))
    
    if(scenario=="Clouds"):

        print("GenerateSpatialDomain - entering clouds generation")

        nCol=spatial_domain_nCol
        nRow=spatial_domain_nRows
        CLOUDS_GENERATOR_TYPE=rand_gen_type
        GRADIENT_TRANSITION_SIZE=clouds_grad_transition
        chaos_attr=None
                
        front_buf,chaos_attr = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr) 
        back_buf,chaos_attr  = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr) 
        back_buf=CorrectWithTransitionMap(GRADIENT_TRANSITION_SIZE, map_1 = front_buf, map_2 = back_buf)
        
        #start generation of the inputs
        cnt=0
        for i in range(0,time_series_length):
            #first we place the cloud over the spatial domain (e.g. over graph)
            
            if(cnt > nRow + GRADIENT_TRANSITION_SIZE):
                sz_front_buf=np.shape(front_buf)
                front_buf= back_buf[0:sz_front_buf[0]][:]
                back_buf,chaos_attr = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr) #nlmpy.mpd(nRow=50, nCol=50, h=0.75)
                back_buf=CorrectWithTransitionMap(GRADIENT_TRANSITION_SIZE, map_1 = front_buf, map_2 = back_buf)
                cnt=0
            mat_show=StickImages(step=cnt, map_1 = front_buf, map_2 = back_buf)
            cnt=cnt+1
            
            for j in range(0,len(nodes)):
                #original coordinates
                pos_x=nodes_X[j]
                pos_y=nodes_Y[j]    
                #tranlated into spatial domain 
                map_x=int((pos_x-bx)/ax)
                map_y=int((pos_y-by)/ay)

                if(map_x>=nRow):
                    map_x=nRow-1
                if(map_y>=nCol):
                    map_y=nCol-1

                rain_input_value=mat_show[map_x,map_y]
                rain_node_input[j].append(rain_input_value)

            if(save_history==True):
                #save mat_show               
                df = pd.DataFrame(mat_show)
                path_to_save_full=clouds_data+"\\cloud_time_step_"+str(i)+".csv"
                df.to_csv(path_to_save_full, header=False, index=False, sep=' ')

        #save nodes input history
        if(save_history==True):            
            #save nodes intake
            for j in range(0,len(nodes)):
                full_path_file=time_series_intake+"\\"+str(nodes[j])+"_rain_input.csv"
                df = pd.DataFrame(rain_node_input[j])
                df.to_csv(full_path_file, header=False, index=False, sep =' ')
                
        return rain_node_input

    if(scenario == "Clouds+terrain"):
        
        #terrains and its watersheds segmentations
        # watersheds maps can be gererated using watershed algorithm for segmentation
        terrain_map,tmp = Terrain_Gen("nlmpy",nRow=nRow, nCol=nCol, h=h_terrain)
        l1,l2=WatershedsSeparation(terrain_map,levels=4) #l1 is the small scale, and l2 is the merged version of l1 with levels, given by user
        
        if(save_history==True):
            #save as scv file
            path_terrain=terrain_data+"\\original_terrain.csv"
            df = pd.DataFrame(terrain_map)
            df.to_csv(path_terrain, header=False, index=False, sep =' ')
            #labbeled terrain with watersheds
            path_terrain_watershed=terrain_data+"\\watershed_retailed.csv"
            df = pd.DataFrame(l1)
            df.to_csv(path_terrain_watershed, header=False, index=False, sep =' ')
            #merged watershed
            path_terrain_watershed_leveled=terrain_data+"\\watershed_user_leveled.csv"
            df = pd.DataFrame(l2)
            df.to_csv(path_terrain_watershed_leveled, header=False, index=False, sep =' ')
            
            #save as picture
            path_terrain_figure=terrain_data+"\\terrain_pictures.jpg"
            fig3,(ax1,ax2,ax3) = plt.subplots(1,3)
            #ax1,ax2 = fig3.add_subplot(211)
            ax1.imshow(terrain_map)            
            ax2.imshow(l1, cmap='viridis', interpolation='nearest')            
            ax3.imshow(l2, cmap='viridis', interpolation='nearest')            
            #plt.show(block=False)
            plt.savefig(path_terrain_figure)
            plt.close()
            
        nCol=spatial_domain_nCol
        nRow=spatial_domain_nRows
        CLOUDS_GENERATOR_TYPE=rand_gen_type
        GRADIENT_TRANSITION_SIZE=clouds_grad_transition
        chaos_attr=None
            
        front_buf,chaos_attr = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr) 
        back_buf,chaos_attr  = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr) 
        back_buf=CorrectWithTransitionMap(GRADIENT_TRANSITION_SIZE, map_1 = front_buf, map_2 = back_buf)
            
        #start generation of the inputs
        cnt=0
        for sim_time_step in range(0,time_series_length):
            
            #first we place the cloud over the spatial domain (e.g. over graph)              
            if(cnt > nRow + GRADIENT_TRANSITION_SIZE):
                sz_front_buf=np.shape(front_buf)
                front_buf= back_buf[0:sz_front_buf[0]][:]
                back_buf,chaos_attr = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr) #nlmpy.mpd(nRow=50, nCol=50, h=0.75)
                back_buf=CorrectWithTransitionMap(GRADIENT_TRANSITION_SIZE, map_1 = front_buf, map_2 = back_buf)
                cnt=0
                
            mat_show=StickImages(step=cnt, map_1 = front_buf, map_2 = back_buf)
            cnt=cnt+1
            
            #----------------------------------------------------------------------
            #compute how much rain went inside each segment (e.g. watershed)
            #----------------------------------------------------------------------
            labels=[]
            water_ccum=[]
            #unique_labels=np.unique(l1)
            for i in range(0,nRow):
                for j in range(0,nCol):
                    label=l1[i,j]
                    if (label in labels)==False:
                        labels.append(label)
                        water_ccum.append(mat_show[i,j])
                    else:
                        index = labels.index(label)
                        water_ccum[index]=water_ccum[index] + mat_show[i,j]

            if(save_history==True):
                map_watersheds_accumulation_water=np.zeros((nRow,nCol))
                for i in range(0,nRow):
                   for j in range(0,nCol):  
                       label=l1[i,j]
                       index = labels.index(label)
                       map_watersheds_accumulation_water[i,j] = water_ccum[index]
                #now we save this as CSV file
                df = pd.DataFrame(map_watersheds_accumulation_water)
                path_to_save_full=terrain_water_accum+"\\watershed_accum_map_time_stamp_"+str(sim_time_step)+".csv"
                df.to_csv(path_to_save_full, header=False, index=False, sep=' ')
        
            #----------------------------------------------------------------------
            #-------------------------------------------
            #----------------------------------------------------------------------
                
            for j in range(0,len(nodes)):
                 #original coordinates
                 pos_x=nodes_X[j]
                 pos_y=nodes_Y[j]    
                 #tranlated into spatial domain 
                 map_x=int((pos_x-bx)/ax)
                 map_y=int((pos_y-by)/ay)
    
                 if(map_x>=nRow):
                     map_x=nRow-1
                 if(map_y>=nCol):
                     map_y=nCol-1               
        
                 label=l1[map_x,map_y]
                 index = labels.index(label)
                 rain_input_value=water_ccum[index]
                 rain_node_input[j].append(rain_input_value)
    
            if(save_history==True):                 
                 #save mat_show  which is the current cloud             
                 df = pd.DataFrame(mat_show)
                 path_to_save_full=clouds_data+"\\cloud_time_step_"+str(sim_time_step)+".csv"
                 df.to_csv(path_to_save_full, header=False, index=False, sep=' ')
    
        #save nodes input history
        if(save_history==True):            
             #save nodes intake
             for j in range(0,len(nodes)):
                 full_path_file=time_series_intake+"\\"+str(nodes[j])+"_rain_input.csv"
                 df = pd.DataFrame(rain_node_input[j])
                 df.to_csv(full_path_file, header=False, index=False, sep =' ')
                
        return rain_node_input

In [ ]:
class MainWindow(QtWidgets.QMainWindow, ui_MainWindow.Ui_ui_MainWindow):
    
    def __init__(self):  
        
        super(MainWindow, self).__init__()
        self.setupUi(self)
        self.init_ui()

        ##pipeline graphs
        self.Graph=None
        self.Inp_Model_Path=None
        #graph data
        self.Node_Names=self.Nodes_coord_X=self.Nodes_coord_Y=None
        self.Cond_Names=self.Nodes_in=self.Nodes_out=self.Length=self.Roughness=None
        self.Junc_Names=self.Junc_elevation=self.Junc_max_depth=self.Junc_init_depth=None
        self.Storage_name=self.Storage_elevation=self.Storage_max_depth=None
        self.Pump_name=self.Pump_From_Node= self.Pump_To_Node=self.Pump_Curve=self.Pump_Status=self.Pump_Sartup=self.Pump_Shutoff=None
        #size of the spatial domain, either for clouds or terrain
        self.nRow=50
        self.nCol=50
        #WNTR simulations
        self.WNTR_results=self.WNTR_nodes=self.WNTR_links=self.WnModel=None
        #EpyT data
        self.EpyT_model=self.EpyT_results=None
        
        #TSNet result data
        self.tsnet_results_nodes_head=[]
        self.tsnet_results_nodes_emit_discharge=[]
        self.tsnet_results_nodes_demand_discharge=[]
                    
        self.tsnet_results_start_head=[]
        self.tsnet_results_end_head=[]
        self.tsnet_results_start_velocity=[]
        self.tsnet_results_end_velocity=[]
        self.tsnet_results_start_flowrate=[]
        self.tsnet_results_end_flowrate=[]
        self.tsnet_node_names=[]
        self.tsnet_pipe_names=[]
                
        #this is for graph network
        self.Plot_Graph_Canvas= MplCanvas(self,width=5, height=4, dpi=100)
        self.horizontalLayout_3.addWidget(self.Plot_Graph_Canvas)
        
        #folders for simulations
        self.Sim_ub_dir_clouds_history="Clouds"
        self.Sim_sub_dir_terrain_hist="Terrain"
        self.Sim_sub_dir_time_series_nodes_intake="Time_series_nodes_intake"
        self.Sim_ub_dir_nodes_params="Nodes_params"

        #time series for clouds
        self.Clouds_timeseries_from_file=None
        
        #global simulation settings
        self.sim_steps_num=None      
        self.sim_steps_units=None      
        self.terrain_max_h=None
        self.max_rain_v=None
        self.waterIntakeFlag=None
        self.waterIntakeScenario=None
        self.savealldata=None
        self.path_save_all_data=None
        #randomized engines fro clouds and terrain generation methods
        self.clouds_gen_methods=None
        self.clouds_terrain_gen_scenario=None
        
        self.time_series=None
        self.input_nodes_names=None
        
        #pollutions
        self.Pollution_introduction_Flag=None
        self.Polluted_node_names=None
        self.Pollution_intrroduction_timestamp=None
        self.Pollution_duration=None
        self.Pollution_level=None
        self.Pollution_injection_params=None
        self.Pollution_timeseries=None
        
    def init_ui(self):
        
        #front panel
        self.ExitButton.clicked.connect(self.Classifier_exit_btn_click)
        self.listWidget.itemClicked.connect(self.SelectElementInGraph)    
        #inp Projects panel
        self.OpenInpProjectButton.clicked.connect(self.Open_inp_btn_click)   
        self.OpenInpProjectButton_2.clicked.connect(self.Open_grap_in_scatter_plot)  
        self.Clouds_loadtimeseriesfromfile_pushButton_11.clicked.connect(self.OpebCloudsTimeSer)        
        #Open clouds
        self.Listallnodes_btnclick.clicked.connect(self.List_inp_model_elements_click)        
        #WNTR simulator
        self.WNTR_simulate_pushButton_3.clicked.connect(self.WNTR_simulate)
        self.WNTR_show_elevations_pushButton_4.clicked.connect(self.WNTR_show_elevation)
        self.WNTR_show_popiulation_pushButton_5.clicked.connect(self.WNTR_show_population)
        self.WNTR_list_WNTR_NODES_AND_PIPES_pushButton_6.clicked.connect(self.WNTR_show_pipes_nodes_click)
        self.WNTR_list_WNTR_SHOW_RESULTSpushButton_7.clicked.connect(self.WNTR_show_sim_results_for_node_click)
        self.WNTRShowMapForTimeStamp_click.clicked.connect(self.WNTR_show_sim_map_click)
        self.WNTR_save_path_browse_pushButton_6.clicked.connect(self.WNTR_browse_save_folder_click)      
        self.WNTR_saveasinopmodel_click_2.clicked.connect(self.WNTR__save_graphj_into_inp_click) 
        #EpyT simulator
        self.EpyT_simulate_pushButton_5.clicked.connect(self.EpyT_simulator_click)
        self.EpyT_showmodel_pushButton_6.clicked.connect(self.EpyT_showmodel_click)
        self.EpyT_showmodelelevation_pushButton_7.clicked.connect(self.EpyT_showmodelelevation_click)
        self.EpyT_showmodelresultsfortiemstamp_pushButton_8.clicked.connect(self.EpyT_showsimulationmapfortimnestamp_click)
        self.EpyT_showmodelresultsfortiemstamp_pushButton_9.clicked.connect(self.EpyT_showsimtimeseriesfornodeslinks_click)
        self.EpyT_listnodelelevations_pushButton_8.clicked.connect(self.EpyT_listnodeselevations_click)
        #TSNet simulation transinet model (based on WNTR model)
        self.TSNet_simulate_pushButton_4.clicked.connect(self.TSNet_simulation_click)
        self.TSNet_showmodelresultsfortiemstamp_pushButton_10.clicked.connect(self.TSNet_shownoderesults_click)
        self.TSNet_resultstimeseries_pushButton_11.clicked.connect(self.TSNet_listtimeseries_click)
        
        
    #front panel
    def List_inp_model_elements_click(self):
        option=self.ListOptionsCombo_2.currentText()
        print("")
        print("")        
        print("Request for " + str(option))
        if(option=="Conduits"):
            if(self.Cond_Names==None):
                print("Conduit list is empty")
            else:
                print("**********************")
                print("******COONDUITS*******")
                print("**********************")                
                for k in range(0,len(self.Cond_Names)):
                    print(self.Cond_Names[k])
                print("")
                print("Conduits number: "+str(len(self.Cond_Names)))
                print("")
        if(option=="Junctions"):
            if(self.Junc_Names==None):
                print("Junction list is empty")
            else:
                print("**********************")
                print("******JUNCTIONS*******")
                print("**********************")        
                for k in range(0,len(self.Junc_Names)):
                    print(self.Junc_Names[k])
                print("")
                print("Junctions number: "+str(len(self.Junc_Names)))
                print("")
        if(option=="Nodes"):
            if(self.Node_Names==None):
                print("Node list is empty")
            else:
                print("**********************")
                print("******Nodes***********")
                print("**********************")        
                for k in range(0,len(self.Node_Names)):
                    print(self.Node_Names[k])
                print("")
                print("Nodes number: "+str(len(self.Node_Names)))
                print("")
        if(option=="Storages"):
            if(self.Storage_name==None):
                print("Storage list is empty")
            else:
                print("**********************")
                print("******Storages********")
                print("**********************")        
                for k in range(0,len(self.Storage_name)):
                    print(self.Storage_name[k])
                print("")
                print("Storages number: "+str(len(self.Storage_name)))
                print("")    
    
    def Classifier_exit_btn_click(self):
        sys.exit(app.exec())

    def SelectElementInGraph(self):
        item=self.listWidget.currentItem()
        if (item is None):
             print("Select file first")
             return
        else:
             item=item.text()     

        if(self.pos2 == None):
            return
        
        self.Plot_Graph_Canvas.highlight_element(self.Graph,self.pos2,highlight_name=item)
            
    #tab - "*.inp files"
    def Open_grap_in_scatter_plot(self):
        if(self.Graph is None):
            print("Load graph first")
            return
            
        %matplotlib qt 
        f1 = plt.figure()
        #nx.draw(self.Graph)
        nx.draw(self.Graph, pos=self.pos2, with_labels=True)
        plt.show()
        %matplotlib inline 
    
    def Open_inp_btn_click(self):
        
        fileNames=""
        dialog = QtWidgets.QFileDialog(self)
        dialog.setFileMode(QtWidgets.QFileDialog.AnyFile)
        dialog.setNameFilter(str("Inp (*.inp)"))
        dialog.setViewMode(QtWidgets.QFileDialog.Detail)
        
        if dialog.exec():
            fileNames = dialog.selectedFiles()
        #try:
            L1,L2,L3,L4,L5,L6,L7,L8,L8_1,L8_2,L8_3,L8_4,L8_5,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21,L22,L23,L24,L25=utr.OpenInp(fileNames[0])
            
            self.Node_Names=L1
            self.Nodes_coord_X=L2
            self.Nodes_coord_Y=L3
            self.Cond_Names=L4
            self.Nodes_in=L5
            self.Nodes_out=L6
            self.Length=L7
            self.Roughness=L8
            self.Conduits_shape=L8_1
            self.Conduits_cross_sect_geom_1=L8_2
            self.Conduits_cross_sect_geom_2=L8_3
            self.Conduits_cross_sect_geom_3=L8_4
            self.Conduits_cross_sect_geom_4=L8_5
            self.Junc_Names=L9
            self.Junc_elevation=L10
            self.Junc_max_depth=L11
            self.Junc_init_depth=L12
            self.Storage_name=L13
            self.Storage_elevation=L14
            self.Storage_max_depth=L15
            self.Pump_name=L16
            self.Pump_From_Node=L17
            self.Pump_To_Node=L18
            self.Pump_Curve=L19
            self.Pump_Status=L20
            self.Pump_Sartup=L21
            self.Pump_Shutoff=L22
            self.Outfalls_name=L23
            self.Outfalls_elevation=L24
            self.Outfalls_type=L25

        #print(len(L4))
        #print(len(L8_1))
            
        #except Exception as X:
        #    print("Error while opening file. Exception rainsed: "+str(X))
        #    return

        print("")
        print("-----------------------------------")
        print("---------*.inp File opened---------")
        print("-----------------------------------")
        print("")
        print("File name: "+str(fileNames))
        print("Nodes num.: "+str(len(self.Node_Names)))
        print("Counduits num.: "+str(len(self.Cond_Names)))
        print("Junctions num.:"+str(len(self.Junc_Names)))
        print("Storages num.:" +str(len(self.Storage_name)))
        print("Pumps num.:" +str(len(self.Pump_name)))
        print("Outfalls num.:" +str(len(self.Outfalls_name)))
        print("")
        print("-----------------------------------")
        print("-----------------------------------")

        #add stuff to list
        self.listWidget.clear()
        for i in range(0,len(self.Cond_Names)):
            #newItem = QListWidgetItem()
            #newItem.setText(itemText)
            self.listWidget.addItem(str(self.Cond_Names[i]))
        #show graph
        self.Graph,self.points,self.pos2=MakeGraph(self.Node_Names,
                                                   self.Nodes_coord_X,
                                                   self.Nodes_coord_Y,
                                                   self.Cond_Names,
                                                   self.Nodes_in,
                                                   self.Nodes_out,
                                                   self.Length,
                                                   self.Roughness,
                                                   self.Conduits_shape,
                                                   self.Conduits_cross_sect_geom_1,
                                                   self.Conduits_cross_sect_geom_2,
                                                   self.Conduits_cross_sect_geom_3,
                                                   self.Conduits_cross_sect_geom_4,
                                                   self.Junc_Names,
                                                   self.Junc_elevation,
                                                   self.Junc_max_depth,
                                                   self.Junc_init_depth,
                                                   self.Storage_name,
                                                   self.Storage_elevation,
                                                   self.Storage_max_depth,
                                                   self.Pump_name,
                                                   self.Pump_From_Node,
                                                   self.Pump_To_Node,
                                                   self.Pump_Curve,
                                                   self.Pump_Status,
                                                   self.Pump_Sartup,
                                                   self.Pump_Shutoff,
                                                   self.Outfalls_name,
                                                   self.Outfalls_elevation,
                                                   self.Outfalls_type
                                                  )
        
        self.Plot_Graph_Canvas.show_graph(self.Graph,self.pos2)
        self.Inp_Model_Path=fileNames       

    #clouds tab settings
    def OpebCloudsTimeSer(self):
        fileNames=""
        dialog = QtWidgets.QFileDialog(self)
        dialog.setFileMode(QtWidgets.QFileDialog.AnyFile)
        dialog.setNameFilter(str("csv (*.csv)"))
        dialog.setViewMode(QtWidgets.QFileDialog.Detail)
        
        if dialog.exec():
            fileNames = dialog.selectedFiles()          
            self.Clouds_Timeseries_text_box_4.setText(fileNames[0])
            df = pd.read_csv(fileNames[0], sep=' ', header=None)
            self.Clouds_timeseries_from_file=[]
            self.Clouds_timeseries_from_file.append(np.asarray(df.values))
            print("")
            print("Successfully opened clouds time series....")
            print("")
    

    #_____________________________________
    #____________________________________
    #---------------------------------------------------------------------------------------------------------
    # before simulatons we read the global settings and generate the corresponding time_series
    #---------------------------------------------------------------------------------------------------------
    #____________________________________
    #_____________________________________
    
    def ReadGlobalSimuationSettings(self):

        if(self.Inp_Model_Path is None) or (): 
            print("Load .inp model first and repeat...") 
            return False

        self.sim_steps_num=int(self.SimSteps_text_box.text())        
        self.sim_steps_units=self.Pywr_duration_units_combo.currentText()    
        self.terrain_max_h=float(np.max(np.asarray(self.Junc_elevation))-np.min(np.asarray(self.Junc_elevation)))
        self.max_rain_v=float(self.WNTR_MaxRain_values_text_box_4.text())
        self.waterIntakeFlag=self.WNTR_waterintakeflag.checkState()        
        self.waterIntakeScenario=self.comboBox.currentText()
        self.savealldata=self.Savealldatacheckbox.checkState()
        self.path_save_all_data=self.WNTR_SimSave_text_box_2.text() 
        self.clouds_terrain_gen_scenario=self.comboBox.currentText()
        self.clouds_gen_methods=self.CloudsGenMethods_comboBox_3.currentText()

        #initialize
        if(self.savealldata==Qt.Checked):   self.savealldata=True
        else:  self.savealldata=False
        if(self.waterIntakeFlag==Qt.Checked): self.waterIntakeFlag=True
        else:  self.waterIntakeFlag=False
            
        #check directories
        if(self.savealldata==True):
            now = datetime.now()
            dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
            self.path_save_all_data=self.path_save_all_data+"\\Exp_"+dt_string
            if not os.path.exists(self.path_save_all_data):
                os.makedirs(self.path_save_all_data)

        #jenerate water injection for the 
        self.time_series=None
        if(self.waterIntakeFlag==True):
            
            if( len(self.Node_Names) == 0):
                print("impossible to generate the spatial domain - graph nodes list is empty")

            print("")    
            print("PARAMETERS OF SPATIAL DOMAIN GENERATION: ")
            print("scenario - "+str(self.clouds_gen_methods))
            print("nodes num. - "+str(len(self.Node_Names)))
            print("spat. domain size - "+str(self.nRow)+"x"+str(self.nCol))
            print("spat. domain rand. func. - "+str(self.clouds_gen_methods))
            print("clouds max. val. - "+str(self.max_rain_v))
            print("terrain max. val. - "+str(self.terrain_max_h))
            print("save data - "+str(self.savealldata))
            if(self.savealldata==True):
                print("save data (dir) - "+str(self.path_save_all_data))
                print("save data for clouds (sub dir) - "+str(self.Sim_ub_dir_clouds_history))
                print("save data for terrain (sub dir) - "+str(self.Sim_sub_dir_terrain_hist))
                print("save data for nodes intake (sub dir) - "+str(self.Sim_sub_dir_time_series_nodes_intake))
                print("save data for nodes params (sub dir) - "+str(self.Sim_ub_dir_nodes_params))
            print("")            
            
            self.time_series=GenerateSpatialDomain(scenario=self.clouds_terrain_gen_scenario,                                                      
                                                   nodes=self.Node_Names,
                                                   time_series_length=self.sim_steps_num,
                                                   nodes_X=self.Nodes_coord_X,
                                                   nodes_Y=self.Nodes_coord_Y,
                                                   spatial_domain_nRows=self.nRow,
                                                   spatial_domain_nCol=self.nCol,
                                                   rand_gen_type=self.clouds_gen_methods,#"const","nlmpy", "lorenz_attractor", "white_noise","const"
                                                   clouds_max_h=self.max_rain_v,                                             
                                                   terrain_max_h=self.terrain_max_h,
                                                   clouds_grad_transition=6,
                                                   save_history=self.savealldata,
                                                   histry_dir=self.path_save_all_data,
                                                   #this are the subdir names
                                                   sub_dir_clouds_history=self.Sim_ub_dir_clouds_history,
                                                   sub_dir_terrain_hist=self.Sim_sub_dir_terrain_hist,
                                                   sub_dir_time_series_nodes_intake=self.Sim_sub_dir_time_series_nodes_intake,
                                                   sub_dir_nodes_params=self.Sim_ub_dir_nodes_params
                                                  )
            if(self.time_series==None) or (self.time_series==[]) or (len(self.time_series)==0):
                    print("Error during simulation, no spatial dommain was created....")
                    return False
            
            #check which nodes to includes for rain input
            #check to which nodes to apply:
            self.input_nodes_names=[]
            intake_nodes_list=[]
            t_series=[]
            
            nodes_apply=self.comboBox_2.currentText()                
            if(nodes_apply=="Specific"):
                text=self.WNTR_SimSave_text_box_3.text()
                list_n=text.split(",")
                for i in range(o,len(list_n)):
                    if(list_n[i]) in self.Node_Names:
                        index=self.Node_Names.index(list_n[i])
                        intake_nodes_list.append(list_n[i])
                        t_series.append(self.time_series[index])
            elif (nodes_apply=="All"):
                t_series=self.time_series
                intake_nodes_list=self.Node_Names
            self.time_series=t_series
            self.input_nodes_names=intake_nodes_list
        else:
            self.time_series=[]
            for p in self.Node_Names:
                t_s=[]
                for k in range(0,self.sim_steps_num):
                    t_s.append(0)
                self.time_series.append(np.asarray(t_s))
            self.input_nodes_names=self.Node_Names

        #THIS IS REAFDING THE POLLUTION SETTINGS
        
        self.Pollution_introduction_Flag=None
        self.Polluted_node_names=None
        self.Pollution_intrroduction_timestamp=None
        self.Pollution_duration=None
        self.Pollution_timeseries=None
        self.Pollution_level=0

        self.Pollution_level = int(self.Pollution_pollutionlevel_text_box_6.text())
        self.Pollution_introduction_Flag=self.WNTR_waterpollutionflag_2.checkState()  
        if(self.Pollution_introduction_Flag==Qt.Checked):   self.Pollution_introduction_Flag=True
        else:  self.Pollution_introduction_Flag=False

        polluted_nodes_n=self.Poolution_nodenames_text_box_6.text()
        split_node_names=polluted_nodes_n.split(",")
        self.Polluted_node_names=[]
        for i in split_node_names:
            if(i!=""):
                self.Polluted_node_names.append(i)

        self.Pollution_intrroduction_timestamp=[]
        pollution_introduction_start = self.Pollution_introductiontimestamp_text_box_7.text()
        split_pollut_intro=pollution_introduction_start.split(",")
        for i in split_pollut_intro:
            if(i!=""):
                self.Pollution_intrroduction_timestamp.append(int(i))
        
        pollution_introduction_start = (self.Pollution_duration_text_box_8.text())
        split_pollut_durat=pollution_introduction_start.split(",")
        self.Pollution_duration=[]
        for i in split_pollut_durat:
            if(i!=""):
                self.Pollution_duration.append(int(i))
        
        #here we generate pollution time series        
        pollution_pattern_shape=nodes_apply=self.CloudsGenMethods_comboBox_4.currentText()           
        self.Pollution_timeseries=[]
        self.Pollution_injection_params=[]
        cnt=0
        for m in self.Polluted_node_names:
            pollut_time_series=[]            
            for k in range(0,self.sim_steps_num):
                if(pollution_pattern_shape=="Rect"): 
                    value=0
                    if(k < int(self.Pollution_intrroduction_timestamp[cnt])):
                        value=0 #pollut_time_series.append(0)
                    if(k>=int(self.Pollution_intrroduction_timestamp[cnt]) and k<=(int(self.Pollution_intrroduction_timestamp[cnt])+int(self.Pollution_duration[cnt]))):
                        value=int(self.Pollution_level) #pollut_time_series.append(int(self.Pollution_level))
                    if(k>int(self.Pollution_intrroduction_timestamp[cnt])+int(self.Pollution_duration[cnt])):
                        #pollut_time_series.append(0)
                        value=0
                    pollut_time_series.append(value)
                                        
                
            self.Pollution_timeseries.append(pollut_time_series)          
            #add parrameters directly
            self.Pollution_injection_params.append(self.Pollution_intrroduction_timestamp[cnt])
            self.Pollution_injection_params.append(int(self.Pollution_intrroduction_timestamp[cnt])+(int(self.Pollution_duration[cnt])))
            self.Pollution_injection_params.append((int(self.Pollution_duration[cnt])))
                        
            cnt=cnt+1
     
        
        if(self.Pollution_introduction_Flag==True):
            print("pollution introdcution - "+str(self.Pollution_introduction_Flag))
            print("pollution introduction nodes num - "+str(len(self.Polluted_node_names)))
            
        return True
        
    #______________________________
    #___TSNet_simulation_click_____
    #______________________________

    def TSNet_simulation_click(self):
        #https://tsnet.readthedocs.io/en/latest/examples.html
        result=self.ReadGlobalSimuationSettings()
        
        if(result==False):
            print("check initialization parameters and repeat...")
            return
            
        #create WNTR model
        """
        wn_model,nodes_n,links_n = WNTR_show_elevation_for_model(G=self.Graph,
                                                                   show_elevation=False,
                                                                   show_population=False,
                                                                   useWaterInputs=self.waterIntakeFlag, 
                                                                   NodeNames=self.input_nodes_names, 
                                                                   PatternsTimeSerries=self.time_series
                                                                  )
        #write inp file       
        current_dir = os.getcwd()
        
        """
        
        
        
        """
        #create EpyT model
        epyT_network,nodes_names,pipe_names=EpyT_makeNet(self.Graph,
                                                    NodeNames=self.Node_Names,
                                                    useWaterInputs=self.waterIntakeFlag,
                                                    PatternsTimeSerries=self.time_series)
                
        model_wntr,n_names,l_names = WNTR_show_elevation_for_model(G=self.Graph,show_elevation=False,show_population=False)
        current_dir = os.getcwd()
        inp_file = current_dir+"\\tmp_epyt_model.inp"
        epyT_network.saveInputFile(inp_file)
        #reload model to TSNet
        tm = tsnet.network.TransientModel(inp_file)
        os.remove(inp_file)
        print("TSNet - model is successfully loaded")
        #apply the correction to the model (introducing the pollution pulses)
       
        if(self.Pollution_introduction_Flag==True):
            for k in self.Polluted_node_names:
                indx=self.Polluted_node_names.index(k)
                tc = self.Pollution_duration[indx] # total demand period [s]
                ts = self.Pollution_intrroduction_timestamp[indx] # demand pulse start time [s]
                tp = 0.2 # demand pulse increase time [s] - we set the increase in dosens of a second
                dp = self.Pollution_level # demand pulse increase multiples [s]
                demand_pulse = [tc,ts,tp,dp]
                tm.add_demand_pulse(k,demand_pulse)
        
        #make settings for the 
        tm.set_wavespeed(1200.)
        #sim.duration
        tf = self.sim_steps_num
        dt = float(self.Pollution_pollutionlevel_text_box_7.text()) #0.1  # time step [s], if not given, use the maximum allowed dt
        tm.set_time(tf,dt)#,dt)
        engine = 'DD' # or PPD
        tm = tsnet.simulation.Initializer(tm,0.,engine)
        results_obj = 'Tsnet_results'
        tm = tsnet.simulation.MOCSimulator(tm,results_obj)
        print("TSNet - simulation is done")
        self.TSNet_results = results_obj
        
        #fill the lists          
        self.TSNeT_node_list_comboBox_4.clear()
        self.TSNet_link_list_comboBox_5.clear()
        #fill the nodes combo
        for t in self.Node_Names:
            self.TSNeT_node_list_comboBox_4.addItem(str(t))
        for m in self.Cond_Names:
            self.TSNeT_node_list_comboBox_4.addItem(str(m))         
        """
        
        current_dir = os.getcwd()
        model_path = current_dir+"\\tmp_wntr_model.inp"
        model_wntr,n_names,l_names = WNTR_show_elevation_for_model(G=self.Graph,show_elevation=False,show_population=False)        
        wntr.network.io.write_inpfile (model_wntr,model_path)    
        tm = tsnet.network.TransientModel(model_path)#'C:\\Users\\shse\\Documents\\ANACONDA\\UTwin_notebooks\\UTWIN_notebooks\\EpyT_init_model.inp')
        os.remove(model_path)
        WAVE_SPEED=float(self.Pollution_pollutionvelocity_text_box_8.text())
        
        nodes_list= tm.node_name_list
        pipes_list= tm.link_name_list
                        
        results_nodes_head=[]
        results_nodes_emit_discharge=[]
        results_nodes_demand_discharge=[]
        
        for l in nodes_list:
            results_nodes_head.append(0)
            results_nodes_emit_discharge.append(0)
            results_nodes_demand_discharge.append(0)
            
        results_start_head=[]
        results_end_head=[]
        results_start_velocity=[]
        results_end_velocity=[]
        results_start_flowrate=[]
        results_end_flowrate=[]
        
        for l in pipes_list:
            results_start_head.append(0)
            results_end_head.append(0)
            results_start_velocity.append(0)
            results_end_velocity.append(0)
            results_start_flowrate.append(0)
            results_end_flowrate.append(0)
            
        Wave_Speed=WAVE_SPEED#12 #m/s
        t0=0
        tf = 1   
        tm.set_wavespeed(Wave_Speed) # m/s
        tm.set_time(tf)          
        engine = 'DD' #demand driven simulator
        tm = tsnet.simulation.Initializer(tm,t0,engine)
        
        #we go one sec by another
        #https://tsnet.readthedocs.io/_/downloads/en/stable/pdf/
        for i in range(0,self.sim_steps_num):     
        
            print("Simultion progress(s): "+str(i) + " out of "+str(self.sim_steps_num))
            tm.t0=i+1
            
            for l in self.input_nodes_names:
                try:
                    indx=self.input_nodes_names.index(l)
                    junction = tm.get_node(l)
                    junction.demand=junction.demand + self.time_series[indx][i]#junction.level=self.time_series[indx][i]
                except:
                    pass
            
            #tm = tsnet.simulation.Initializer(tm,t0)
            tm = tsnet.simulation.MOCSimulator(tm)
        
            #results nodes
            for l in nodes_list:
                
                node=tm.get_node(l)
                indx=nodes_list.index(l)
                head = np.nan_to_num(node.head)
                emitter_discharge = np.nan_to_num(node.emitter_discharge) #np.nan_to_num(a)
                demand_discharge = np.nan_to_num(node.demand_discharge)     
        
                tmp1=np.asarray(results_nodes_head[indx])
                tmp2=np.concatenate((tmp1, np.asarray(head)), axis=None)
                results_nodes_head[indx]=np.empty
                results_nodes_head[indx]=tmp2
        
                tmp1=np.asarray(results_nodes_emit_discharge[indx])
                tmp2=np.concatenate((tmp1, np.asarray(emitter_discharge)), axis=None)
                results_nodes_emit_discharge[indx]=np.empty
                results_nodes_emit_discharge[indx]=tmp2
        
                tmp1=np.asarray(results_nodes_demand_discharge[indx])
                tmp2=np.concatenate((tmp1, np.asarray(demand_discharge)), axis=None)
                results_nodes_demand_discharge[indx]=np.empty
                results_nodes_demand_discharge[indx]=tmp2
                
                #results links
            for l in pipes_list:
                
                pipe=tm.get_link(l)
                indx=pipes_list.index(l)
                
                start_head = np.nan_to_num(pipe.start_node_head)
                end_head = np.nan_to_num(pipe.end_node_head)
                start_velocity = np.nan_to_num(pipe.start_node_velocity)
                end_velocity = np.nan_to_num(pipe.end_node_velocity)
                start_flowrate = np.nan_to_num(pipe.start_node_flowrate)
                end_flowrate = np.nan_to_num(pipe.end_node_flowrate)
                  
                tmp1=np.asarray(results_start_head[indx])
                tmp2=np.concatenate((tmp1, np.asarray(start_head)), axis=None)
                results_start_head[indx]=np.empty
                results_start_head[indx]=tmp2
        
                tmp1=np.asarray(results_end_head[indx])
                tmp2=np.concatenate((tmp1, np.asarray(end_head)), axis=None)
                results_end_head[indx]=np.empty
                results_end_head[indx]=tmp2
                
                tmp1=np.asarray(results_start_velocity[indx])
                tmp2=np.concatenate((tmp1, np.asarray(start_velocity)), axis=None)
                results_start_velocity[indx]=np.empty
                results_start_velocity[indx]=tmp2
                
                tmp1=np.asarray(results_end_velocity[indx])
                tmp2=np.concatenate((tmp1, np.asarray(end_velocity)), axis=None)
                results_end_velocity[indx]=np.empty
                results_end_velocity[indx]=tmp2
        
                tmp1=np.asarray(results_start_flowrate[indx])
                tmp2=np.concatenate((tmp1, np.asarray(start_flowrate)), axis=None)
                results_start_flowrate[indx]=np.empty
                results_start_flowrate[indx]=tmp2
        
                tmp1=np.asarray(results_end_flowrate[indx])
                tmp2=np.concatenate((tmp1, np.asarray(end_flowrate)), axis=None)
                results_end_flowrate[indx]=np.empty
                results_end_flowrate[indx]=tmp2
                        
        #tm.plot_node_head(L1[0])
        print("")
        print("___________________________")
        print("TSnet - simulation complete:")
        print("Simulation results shape for nodes: " + str(np.shape(np.asarray(results_start_head))))
        print("Simulation results shape for links: " + str(np.shape(np.asarray(results_start_velocity))))
        print("___________________________")
        print("")
            
        #fill globals
        self.tsnet_results_nodes_head=results_nodes_head
        self.tsnet_results_nodes_emit_discharge=results_nodes_emit_discharge
        self.tsnet_results_nodes_demand_discharge=results_nodes_demand_discharge
                    
        self.tsnet_results_start_head=results_start_head
        self.tsnet_results_end_head=results_end_head
        self.tsnet_results_start_velocity=results_start_velocity
        self.tsnet_results_end_velocity=results_end_velocity
        self.tsnet_results_start_flowrate=results_start_flowrate
        self.tsnet_results_end_flowrate=results_end_flowrate

        self.tsnet_node_names=nodes_list
        self.tsnet_pipe_names=pipes_list
        
        #fill the lists          
        self.TSNeT_node_list_comboBox_4.clear()
        self.TSNet_link_list_comboBox_5.clear()
        #fill the nodes combo
        for t in nodes_list:
            self.TSNeT_node_list_comboBox_4.addItem(str(t))
        for m in pipes_list:
            self.TSNet_link_list_comboBox_5.addItem(str(m))         
       
       
    def TSNet_shownoderesults_click(self):
        
        #if(self.TSNET_results == False):
        #    print("Run WNTR simulation first")
        #    return  
            
        node_selected= str(self.TSNeT_node_list_comboBox_4.currentText())
        link_selected= str(self.TSNet_link_list_comboBox_5.currentText())       

        node_index=self.tsnet_node_names.index(node_selected)
        pipe_index=self.tsnet_pipe_names.index(link_selected)
        
        #check what to show
        show_node_head_falg=self.TSNet_noidehead_checkBox_4.checkState()
        if(show_node_head_falg==Qt.Checked):   show_node_head_falg=True
        else:  show_node_head_falg=False

        show_node_emit_falg=self.TSNet_nodeemit_checkBox_5.checkState()
        if(show_node_emit_falg==Qt.Checked):   show_node_emit_falg=True
        else:  show_node_emit_falg=False

        show_node_demand_falg=self.TSNet_demanddischarge_checkBox_6.checkState()
        if(show_node_demand_falg==Qt.Checked):   show_node_demand_falg=True
        else:  show_node_demand_falg=False

        if(show_node_head_falg==True) or (show_node_emit_falg==True) or (show_node_demand_falg==True):
            %matplotlib qt 
            tsnet_fig_results1234_links, ax = plt.subplots()   
            if(show_node_head_falg==True):
                ax.plot(self.tsnet_results_nodes_head[node_index],label="Head") 
            if(show_node_emit_falg==True):
                ax.plot(self.tsnet_results_nodes_emit_discharge[node_index],label="Emitter discharge") 
            if(show_node_demand_falg==True):
                ax.plot(self.tsnet_results_nodes_demand_discharge[node_index],label="Demand discharge") 
            ax.set_title("Node "+ node_selected +" data")
            #wntr.graphics.plot_network(wn, node_attribute='elevation')        
            plt.show()
            %matplotlib inline    

        #check what to show
        pipe_start_head_falg=self.TSNet_starthead_checkBox_5.checkState()
        if(pipe_start_head_falg==Qt.Checked):   pipe_start_head_falg=True
        else:  pipe_start_head_falg=False

        pipe_wnd_head_falg=self.TSNet_endhead_checkBox_6.checkState()
        if(pipe_wnd_head_falg==Qt.Checked):   pipe_wnd_head_falg=True
        else:  pipe_wnd_head_falg=False

        pipe_startvelocity_falg=self.TSNet_startvelociy_checkBox_7.checkState()
        if(pipe_startvelocity_falg==Qt.Checked):   pipe_startvelocity_falg=True
        else:  pipe_startvelocity_falg=False

        pipe_endvelocity_falg=self.TSNet_endvelcity_checkBox_8.checkState()
        if(pipe_endvelocity_falg==Qt.Checked):   pipe_endvelocity_falg=True
        else:  pipe_endvelocity_falg=False

        pipe_startflowrate_falg=self.TSNet_startflow_checkBox_9.checkState()
        if(pipe_startflowrate_falg==Qt.Checked):   pipe_startflowrate_falg=True
        else:  pipe_startflowrate_falg=False

        pipe_endflowrate_falg=self.TSNet_endflow_checkBox_10.checkState()
        if(pipe_endflowrate_falg==Qt.Checked):   pipe_endflowrate_falg=True
        else:  pipe_endflowrate_falg=False
        
        
        if(pipe_start_head_falg) or (pipe_wnd_head_falg) or (pipe_startvelocity_falg) or (pipe_endvelocity_falg) or (pipe_startflowrate_falg) or (pipe_endflowrate_falg):
            
            %matplotlib qt 
            tsnet_fig_results1234_links, ax = plt.subplots()   
            if(pipe_start_head_falg==True):
                ax.plot(self.tsnet_results_start_head[pipe_index],label="Start head") 
            if(pipe_wnd_head_falg==True):
                ax.plot(self.tsnet_results_end_head[pipe_index],label="End head") 
            if(pipe_startvelocity_falg==True):
                ax.plot(self.tsnet_results_start_velocity[pipe_index],label="Start veloity") 
            if(pipe_endvelocity_falg==True):
                ax.plot(self.tsnet_results_end_velocity[pipe_index],label="End velocity") 
            if(pipe_startflowrate_falg==True):
                ax.plot(self.tsnet_results_start_flowrate[pipe_index],label="Start flow") 
            if(pipe_endflowrate_falg==True):
                ax.plot(self.tsnet_results_end_flowrate[pipe_index],label="End flow") 
            ax.set_title("pipe "+ link_selected +" data")
            #wntr.graphics.plot_network(wn, node_attribute='elevation')        
            plt.show()
            %matplotlib inline    
        
        #results are stored here
        
        #fill globals
        """
        self.tsnet_results_nodes_head=results_nodes_head
        self.tsnet_results_nodes_emit_discharge=results_nodes_emit_discharge
        self.tsnet_results_nodes_demand_discharge=results_nodes_demand_discharge
                    
        self.tsnet_results_start_head=results_start_head
        self.tsnet_results_end_head=results_end_head
        self.tsnet_results_start_velocity=results_start_velocity
        self.tsnet_results_end_velocity=results_end_velocity
        self.tsnet_results_start_flowrate=results_start_flowrate
        self.tsnet_results_end_flowrate=results_end_flowrate
        """

    def TSNet_listtimeseries_click(self):
        
        node_selected= str(self.TSNeT_node_list_comboBox_4.currentText())
        link_selected= str(self.TSNet_link_list_comboBox_5.currentText())       

        node_index=self.tsnet_node_names.index(node_selected)
        pipe_index=self.tsnet_pipe_names.index(link_selected)
        
        #check what to show
        show_node_head_falg=self.TSNet_noidehead_checkBox_4.checkState()
        if(show_node_head_falg==Qt.Checked):   show_node_head_falg=True
        else:  show_node_head_falg=False

        show_node_emit_falg=self.TSNet_nodeemit_checkBox_5.checkState()
        if(show_node_emit_falg==Qt.Checked):   show_node_emit_falg=True
        else:  show_node_emit_falg=False

        show_node_demand_falg=self.TSNet_demanddischarge_checkBox_6.checkState()
        if(show_node_demand_falg==Qt.Checked):   show_node_demand_falg=True
        else:  show_node_demand_falg=False

        if(show_node_head_falg==True) or (show_node_emit_falg==True) or (show_node_demand_falg==True):
            
            if(show_node_head_falg==True):
                print("")
                print("__________________________")
                print("______Node____head________")
                print("__________________________")                
                print(self.tsnet_results_nodes_head[pipe_index]) 
                print("")
                print("")               
            if(show_node_emit_falg==True):
                print("")
                print("_______________________________")
                print("______Node____emit discharge___")
                print("_______________________________")                
                print(self.tsnet_results_nodes_emit_discharge[pipe_index]) 
                print("")
                print("")      
                
            if(show_node_demand_falg==True):
                print("")
                print("_______________________________")
                print("______Node____emit discharge___")
                print("_______________________________")                
                print(self.tsnet_results_nodes_demand_discharge[pipe_index]) 
                print("")
                print("")      
                
        #check what to show
        pipe_start_head_falg=self.TSNet_starthead_checkBox_5.checkState()
        if(pipe_start_head_falg==Qt.Checked):   pipe_start_head_falg=True
        else:  pipe_start_head_falg=False

        pipe_wnd_head_falg=self.TSNet_endhead_checkBox_6.checkState()
        if(pipe_wnd_head_falg==Qt.Checked):   pipe_wnd_head_falg=True
        else:  pipe_wnd_head_falg=False

        pipe_startvelocity_falg=self.TSNet_startvelociy_checkBox_7.checkState()
        if(pipe_startvelocity_falg==Qt.Checked):   pipe_startvelocity_falg=True
        else:  pipe_startvelocity_falg=False

        pipe_endvelocity_falg=self.TSNet_endvelcity_checkBox_8.checkState()
        if(pipe_endvelocity_falg==Qt.Checked):   pipe_endvelocity_falg=True
        else:  pipe_endvelocity_falg=False

        pipe_startflowrate_falg=self.TSNet_startflow_checkBox_9.checkState()
        if(pipe_startflowrate_falg==Qt.Checked):   pipe_startflowrate_falg=True
        else:  pipe_startflowrate_falg=False

        pipe_endflowrate_falg=self.TSNet_endflow_checkBox_10.checkState()
        if(pipe_endflowrate_falg==Qt.Checked):   pipe_endflowrate_falg=True
        else:  pipe_endflowrate_falg=False
        
        
        if(pipe_start_head_falg) or (pipe_wnd_head_falg) or (pipe_startvelocity_falg) or (pipe_endvelocity_falg) or (pipe_startflowrate_falg) or (pipe_endflowrate_falg):
                        
            if(pipe_start_head_falg==True):
                print("")
                print("__________________________")
                print("______Pipe__Start head________")
                print("__________________________")                
                print(self.tsnet_results_start_head[pipe_index]) 
                print("")
                print("")
            if(pipe_wnd_head_falg==True):
                print("")
                print("__________________________")
                print("_______Pipe___End head________")
                print("__________________________")    
                print(self.tsnet_results_end_head[pipe_index])
                print("")
                print("")
            if(pipe_startvelocity_falg==True):
                print("")
                print("__________________________")
                print("______Pipe__Start velocity ___")
                print("__________________________")   
                print(self.tsnet_results_start_velocity[pipe_index])
                print("")
                print("")
            if(pipe_endvelocity_falg==True):
                print("")
                print("__________________________")
                print("_______Pipe___End velocity ___")
                print("__________________________")   
                print(self.tsnet_results_end_velocity[pipe_index]) 
                print("")
                print("")                
            if(pipe_startflowrate_falg==True):
                print("")
                print("__________________________")
                print("________Pipe__Start flow__ ___")
                print("__________________________")   
                print(self.tsnet_results_start_flowrate[pipe_index]) 
                print("")
                print("")    
            if(pipe_endflowrate_falg==True):
                print("")
                print("__________________________")
                print("________Pipe__End flow__ ___")
                print("__________________________")   
                ax.plot(self.tsnet_results_end_flowrate[pipe_index],label="End flow") 
                print("")
                print("")   

        
    #______________________________    
    #WNTR simulation
    #______________________________    
    def WNTR__save_graphj_into_inp_click(self):
        pass
    
    def WNTR_browse_save_folder_click(self):
        selected_directory = QtGui.QFileDialog.getExistingDirectory()
        self.WNTR_SimSave_text_box_2.setText(selected_directory)
    
    def WNTR_simulate(self):

        """
        sim_steps=int(self.SimSteps_text_box.text())
        
        sim_dur_type=self.Pywr_duration_units_combo.currentText()
        if(self.Inp_Model_Path is None): 
            print("Load .inp model first and repeat...") 
            return
       
        #read the settings
        waterIntakeFlag=self.WNTR_waterintakeflag.checkState()        
        waterIntakeScenario=self.comboBox.currentText()
        savealldata=self.Savealldatacheckbox.checkState()
        path_save_all_data=self.WNTR_SimSave_text_box_2.text()
        clouds_gen_methods=self.CloudsGenMethods_comboBox_3.currentText()
        
        if(savealldata==Qt.Checked):   savealldata=True
        else:  savealldata=False

        if(waterIntakeFlag==Qt.Checked): waterIntakeFlag=True
        else:  waterIntakeFlag=False
        
        #prepare save folder
        if(savealldata==True):
            now = datetime.now()
            dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
            path_save_all_data=path_save_all_data+"\\Exp_"+dt_string
            if not os.path.exists(path_save_all_data):
                os.makedirs(path_save_all_data)

        terrain_max_h=float(np.max(np.asarray(self.Junc_elevation))-np.min(np.asarray(self.Junc_elevation)))
        max_rain_v=float(self.WNTR_MaxRain_values_text_box_4.text())
        
        intake_nodes_list=[]
        t_series=[]
        
        if(waterIntakeFlag==True):             
                time_series=GenerateSpatialDomain(scenario=waterIntakeScenario,nodes=self.Node_Names,
                                                     time_series_length=sim_steps,
                                                     nodes_X=self.Nodes_coord_X,
                                                     nodes_Y=self.Nodes_coord_Y,
                                                     spatial_domain_nRows=self.nRow,
                                                     spatial_domain_nCol=self.nCol,
                                                     rand_gen_type=clouds_gen_methods,#"const","nlmpy", "lorenz_attractor", "white_noise","const"
                                                     clouds_max_h=max_rain_v,                                             
                                                     terrain_max_h=terrain_max_h,
                                                     clouds_grad_transition=6,
                                                     save_history=savealldata,
                                                     histry_dir=path_save_all_data,
                                                     #this are the subdir names
                                                     sub_dir_clouds_history=self.Sim_ub_dir_clouds_history,
                                                     sub_dir_terrain_hist=self.Sim_sub_dir_terrain_hist,
                                                     sub_dir_time_series_nodes_intake=self.Sim_sub_dir_time_series_nodes_intake,
                                                     sub_dir_nodes_params=self.Sim_ub_dir_nodes_params
                                                 )
            
            
            
            

                if(time_series==None):
                    print("Error during simulation, no spatial dommain was created....")
                    return
            
                #check which nodes to includes for rain input
                #check to which nodes to apply:
                nodes_apply=self.comboBox_2.currentText()
                
                if(nodes_apply=="Specific"):
                    text=self.WNTR_SimSave_text_box_3.text()
                    list_n=text.split(",")
                    for i in range(o,len(list_n)):
                        if(list_n[i]) in self.Node_Names:
                            index=self.Node_Names.index(list_n[i])
                            intake_nodes_list.append(list_n[i])
                            t_series.append(time_series[index])
                            
                elif (nodes_apply=="All"):
                    t_series=time_series
                    intake_nodes_list=self.Node_Names
        """
        
        result=self.ReadGlobalSimuationSettings()
        
        if(result==False):
            print("Settings are incorrect. Check and repeat...")
            return
                
        self.WnModel,self.WNTR_results,self.WNTR_nodes,self.WNTR_links=WNTR(graph=self.Graph,
                                                                            sim_dur=self.sim_steps_num,
                                                                            duration_type=self.sim_steps_units,
                                                                            useWaterInputs=self.waterIntakeFlag, 
                                                                            NodeNames= self.input_nodes_names, 
                                                                            PatternsTimeSerries=self.time_series
                                                                           )
        
        #fill uzp widgets
        self.WNTR_nodes_list_combo.clear()
        self.WNTR_links_list_combo.clear()
        #fill the nodes combo
        for t in self.WNTR_nodes:
            self.WNTR_nodes_list_combo.addItem(str(t))
        for m in self.WNTR_links:
            self.WNTR_links_list_combo.addItem(str(m))            

        if(self.savealldata==True):
            path=path_save_all_data+"\\"+self.Sim_ub_dir_nodes_params+"\\Junctions\\"
            if not os.path.exists(path):
                os.makedirs(path)

            node_keys_access = self.WNTR_results.node.keys()
            keys_nd= list(node_keys_access)[:]
                        
            for node in self.WNTR_nodes:
                node_selected=node

                for gh in keys_nd:

                    path_with_key=path+"\\"+gh
                    if not os.path.exists(path_with_key):
                        os.makedirs(path_with_key)

                    n_data=self.WNTR_results.node[gh].loc[:,node_selected]     
                    path_to_file=path_with_key+"\\node_"+node+"_"+str(gh)+".csv"
                    
                    df = pd.DataFrame(n_data)                    
                    df.to_csv(path_to_file, header=False, index=False, sep=' ')                   
               
    def WNTR_show_elevation(self):        
        if(self.Graph is None):
            print("Graph is empty. Load model first")
            return
        else:
            WNTR_show_elevation_for_model(G=self.Graph,show_elevation=True)
    def WNTR_show_population(self):
        if(self.Graph is None):
            print("Graph is empty. Load model first")
            return
        else:
            wn=WNTR_show_elevation_for_model(G=self.Graph,show_population=True)    
    def WNTR_show_pipes_nodes_click(self):
        
        if(self.WNTR_results==False):
            print("Run WNTR simulation first")
            return  

        print("")
        print("++++++++++++++++++++++++++++++")
        print("WNTR model nodes")
        print("++++++++++++++++++++++++++++++")
        print("")
        for p in self.WNTR_nodes:
            print(p)
            
        print("")
        print("++++++++++++++++++++++++++++++")
        print("WNTR model links")
        print("++++++++++++++++++++++++++++++")
        print("")
        for p in self.WNTR_links:
            print(p)

        print("")
        print("+++++++++++++++++++++++++++++")
        print("End of list")
        print("+++++++++++++++++++++++++++++")

    def WNTR_show_sim_results_for_node_click(self):
        
        if(self.WNTR_results==False):
            print("Run WNTR simulation first")
            return  
            
        node_selected= str(self.WNTR_nodes_list_combo.currentText())
        link_selected= str(self.WNTR_links_list_combo.currentText())
        node_keys_access = self.WNTR_results.node.keys()
        link_keys_access = self.WNTR_results.link.keys()
        
        #print(node_keys)
        node_results=[]
        link_results=[] 
        print("")
        node_legends=[]
        link_legends=[]

        keys_nd= list(node_keys_access)[:]
        link_nd= list(link_keys_access)[:]        
        
        if(node_selected!=""):      
            print("_EXTRACTING DATA FOR NODE: "+node_selected+" ")
            for bs in keys_nd:                                        
                try:                    
                    print("Demand key: "+bs)
                    #results_curr_parameter=self.WNTR_results.node[k]                    
                    n_data=self.WNTR_results.node[bs].loc[:,node_selected]                    
                    node_results.append(n_data)
                    node_legends.append(bs)
                    #print("Data is extracted for the key. Data length: "+str(len(n_data)))
                except Exception as ex:
                    print("Results for key "+bs+" are not present")                
        else:
            print("Node is not selected")           

            
        if(link_selected!=""):
            print("_EXTRACTING DATA FOR LINK: "+ link_selected + " ")      
            for bs in link_nd:
                try:                    
                    n_data=self.WNTR_results.link[bs].loc[:,link_selected]                            
                    link_results.append(n_data) 
                    link_legends.append(bs)
                except Exception as ex:
                    print("Results for key "+bs+" are not present")                  
        else:
            print("Node is not selected")

        demand_flag=self.WNTR_SHowdemand_checkbox_4.checkState()
        head_flag=self.WNTR_SHowhead_checkbox_5.checkState()
        pressure_flag=self.WNTR_SHowpressure_checkbox_6.checkState()
        quality_flag=self.WNTR_SHowquality_checkbox_7.checkState()

        if(demand_flag==Qt.Checked): demand_flag=True
        else: demand_flag=False
        if(head_flag==Qt.Checked): head_flag=True
        else: head_flag=False
        if(pressure_flag==Qt.Checked): pressure_flag=True
        else: pressure_flag=False
        if(quality_flag==Qt.Checked): quality_flag=True
        else: quality_flag=False

        if(demand_flag==True or head_flag==True or pressure_flag==True or quality_flag==True):
            %matplotlib qt 
            WNTR_fig_results1234, ax = plt.subplots()   
            if(demand_flag==True):
                ax.plot(node_results[0])  
                plt.legend(node_legends[0])
            if(head_flag==True):
                ax.plot(node_results[1]) 
                plt.legend(node_legends[1])
            if(pressure_flag==True):
                ax.plot(node_results[2])  
                plt.legend(node_legends[2])
            if(quality_flag==True):
                ax.plot(node_results[3]) 
                plt.legend(node_legends[3])
            #plt.legend(node_legends)    #plt.legend(loc="upper left")
            ax.set_title("Node "+ node_selected +" data")
            #wntr.graphics.plot_network(wn, node_attribute='elevation')        
            plt.show()
            %matplotlib inline 
            #print("Results for node are shown")

        #------------LINKS------------------
        #-----------------------------------
        quality_flag=self.WNTR_SHowlinkquality_checkbox_5.checkState()
        flowrate_flag=self.WNTR_SHowlinkflowrate_checkbox_6.checkState()
        flowvelocity_flag=self.WNTR_SHowlinkvelocity_checkbox_7.checkState()
        headloss_flag=self.WNTR_SHowlinkheadloss_checkbox_8.checkState()
        frictionrate_flag=self.WNTR_SHowlinkfriction_checkbox_9.checkState()
        reatcionrate_flag=self.WNTR_SHowlinkreactionrate_checkbox_10.checkState()

        if(quality_flag==Qt.Checked): quality_flag=True
        else: quality_flag=False
        if(flowrate_flag==Qt.Checked): flowrate_flag=True
        else: flowrate_flag=False
        if(flowvelocity_flag==Qt.Checked): flowvelocity_flag=True
        else: flowvelocity_flag=False
        if(headloss_flag==Qt.Checked): headloss_flag=True
        else: headloss_flag=False
        if(frictionrate_flag==Qt.Checked): frictionrate_flag=True
        else: frictionrate_flag=False
        if(reatcionrate_flag==Qt.Checked): reatcionrate_flag=True
        else: reatcionrate_flag=False

        if(quality_flag==True) or (flowrate_flag==True) or (flowvelocity_flag==True) or (headloss_flag==True) or (frictionrate_flag==True) or (reatcionrate_flag==True):
            
            %matplotlib qt 
            WNTR_fig_results1234_links, ax = plt.subplots()   
            if(quality_flag==True):
                ax.plot(link_results[0],label="Quality") 
            if(flowrate_flag==True):
                ax.plot(link_results[1],label="Flowrate") 
            if(flowvelocity_flag==True):
                ax.plot(link_results[2],label="Velocity") 
            if(headloss_flag==True):
                ax.plot(link_results[3],label="Headloss") 
            if(frictionrate_flag==True):
                ax.plot(link_results[6],label="Friction factor") 
            if(reatcionrate_flag==True):
                ax.plot(link_results[7],label="Reaction rate")                
            ax.set_title("Link " + str(link_selected) + " data")
            #wntr.graphics.plot_network(wn, node_attribute='elevation')        
            plt.show()
            %matplotlib inline 
            #print("Results for links are shown")

    def WNTR_show_sim_map_click(self):
        
        timeStamp=int(self.WNTR_SimSteps_text_box_2.text())
        show_junc=self.Savealldatacheckbox_2.checkState()
        if(show_junc==Qt.Checked): show_junc=True
        else: show_junc=False

        show_pipe=self.Savealldatacheckbox_2.checkState()
        if(show_pipe==Qt.Checked): show_pipe=True
        else: show_pipe=False
            
        #https://github.com/usepa/WNTR/blob/main/documentation/resultsobject.rst#id6
        if(self.WNTR_results==False):
            print("Run WNTR simulation first")
            return  
            
        node_keys_access = self.WNTR_results.node.keys()
        link_keys_access = self.WNTR_results.link.keys()
        keys_nd= list(node_keys_access)[:]
        link_nd= list(link_keys_access)[:]   

        if(show_junc==True):
           for bs in keys_nd:                                        
                try:                
                    #results_curr_parameter=self.WNTR_results.node[k]                    
                    n_data=self.WNTR_results.node[bs].loc[timeStamp,:]                  
                    %matplotlib qt 
                    figure_tmp,ax = plt.subplots() 
                    ax = wntr.graphics.plot_network(self.WnModel, node_attribute=n_data,node_range=[0,len(self.WNTR_nodes)], node_colorbar_label=bs) 
                    #figure_tmp.suptitle(bs, fontsize=16)
                    title=str("Time stamp: "+str(timeStamp)+" , parameters: "+bs)
                    ax.set_title(title)
                    plt.tight_layout()                    
                    #plt.savefig('plot_network.png', dpi=300)
                    #plt.close()
                    plt.show()
                    %matplotlib inline                     
                    time.sleep(0.5)
                except Exception as ex:
                    print("Results for key "+bs+" are not present")  

        if(show_pipe==False):
            for bs in link_nd:                                        
                try:                
                    #results_curr_parameter=self.WNTR_results.node[k]                    
                    n_data=self.WNTR_results.link[bs].loc[timeStamp,:]                  
                    %matplotlib qt 
                    figure_tmp,ax = plt.subplots() 
                    ax = wntr.graphics.plot_network(self.WnModel, node_attribute=n_data,node_range=[0,len(self.WNTR_links)], node_colorbar_label=bs) 
                    #figure_tmp.suptitle(bs, fontsize=16)
                    title=str("Time stamp: "+str(timeStamp)+" , parameters: "+bs)
                    ax.set_title(title)
                    plt.tight_layout()                    
                    plt.show()
                    %matplotlib inline                     
                    time.sleep(0.5)
                except Exception as ex:
                    print("Results for key "+bs+" are not present")      


        #___________________________________________________
        #___panda pipe______________________________________
        #_____simulate_________________________________________

    def Pandapipe_simulate_click(self):
        #simulations with pandapipes
        sim_steps=int(self.SimSteps_text_box.text())        
        if(self.Inp_Model_Path is None): 
            print("Load .inp model first and repeat...") 
            return
        #open model with pandaspipe
        #https://www.pandapipes.org/
        ppModel,nodes_names,pipes_names=Pandapipes_makeNet(graph=self.Graph)

    #___________________________________________________
    #___EpyT simulator__________________________________
    #_____simulate______________________________________
    
    def EpyT_listnodeselevations_click(self):
        if(self.EpyT_model==None):
            print("Run simulation first")
            return
        d=self.EpyT_model
        nodes_n=d.getNodeNameID()
        print("")
        print("Nodes elevations (m over see level): ")
        print("")
        for i in range(0,len(nodes_n)):
            elev = d.getNodeElevations(i)
            print("Node "+str(nodes_n[i])+"elevation -> "+str(elev))
    
    def EpyT_showsimtimeseriesfornodeslinks_click(self):
        
        if(self.EpyT_model==None):
            print("Run simulation first")
            return
        d=self.EpyT_model

        nodes_n=d.getNodeNameID()
        links_n=self.EpyT_model.getNodesConnectingLinksID()
        
        selected_node_name=str(self.EpyT_node_list_comboBox_3.currentText())
        selected_link_name=str(self.EpyT_link_list_comboBox_4.currentText())   
        link_indx=-1
        cnt=0
        
        for z in links_n:       
            if(z[0] in selected_link_name)==True and (z[1] in selected_link_name)==True:            
                link_indx=cnt                         
                break
            cnt=cnt+1
        
        node_indx=list(nodes_n).index(selected_node_name)
        node_pressure=self.EpyT_results[1]
        node_demand=self.EpyT_results[2]
        node_overhead=self.EpyT_results[3]
        link_flow=self.EpyT_results[4]
        link_velocity=self.EpyT_results[5]
        link_overhead=self.EpyT_results[6]
        
        #print("NODE PRESSURE SIZE: "+str(np.shape(node_pressure)))
        node_pressure=np.asarray(node_pressure)
        node_pressure=node_pressure[:,node_indx]

        node_demand=np.asarray(node_demand)
        node_demand=node_demand[:,node_indx]

        node_overhead=np.asarray(node_overhead)
        node_overhead=node_overhead[:,node_indx]

                
        #print("LINK FLOW SIZE: "+str(np.shape(link_flow)))
        link_flow=np.asarray(link_flow)
        link_flow=link_flow[:,link_indx]

        link_velocity=np.asarray(link_velocity)
        link_velocity=link_velocity[:,node_indx]

        link_overhead=np.asarray(link_overhead)
        link_overhead=link_overhead[:,node_indx]
        
        #this is how results are packed -  self.EpyT_results = [T,P,D,H,F,V,LH]
        #check the nodes inputs        
        index=-1
        try:
            index=list(self.EpyT_Nodes_Names_Inputs).index(selected_node_name)
        except:
            pass
        NodeInput=None
        if(index!=-1):
            NodeInput=self.EpyT_Nodes_Inputs[index]

        #nodes
        pressureflag=self.EpyT_shownodepressure_checkBox_3.checkState()
        demandflag=self.EpyT_shownodedemand_checkBox_4.checkState()
        headflag=self.EpyT_shownodehead_checkBox_5.checkState()
        inputflag=self.EpyT_shownodeinput_checkBox_6.checkState()
        
        if(pressureflag==Qt.Checked):   pressureflag=True
        else:  pressureflag=False
        if(demandflag==Qt.Checked):   demandflag=True
        else:  demandflag=False
        if(headflag==Qt.Checked):   headflag=True
        else:  headflag=False        
        if(inputflag==Qt.Checked):   inputflag=True
        else:  inputflag=False        
        
        
        if(pressureflag==True) or (demandflag==True) or (headflag==True) or (inputflag==True):
            %matplotlib qt 
            figure_tmp,ax = plt.subplots() 
            if(pressureflag):
                plt.plot(node_pressure,color='green',label=str("Node "+str(selected_node_name)+" pressure"))        
            if(NodeInput is None)==False and inputflag==True:
                plt.plot(NodeInput,color='red',label=str("Node "+str(selected_node_name)+" input"))  
            if(demandflag):
                plt.plot(node_demand,color='blue',label='Node demand')  
            if(headflag):
                plt.plot(node_overhead,color='orange',label='Node overhead')        
            plt.legend()
            plt.tight_layout()                    
            plt.show()
            %matplotlib inline                     
            time.sleep(0.5)
        
        flowflag=self.EpyT_showlinkflow_checkBox_2.checkState()
        velocityflag=self.EpyT_showlinkvelocity_checkBox.checkState()
        headflag=self.EpyT_showolinkverheadcheckBox_2.checkState()
        if(flowflag==Qt.Checked):   flowflag=True
        else:  flowflag=False
        if(velocityflag==Qt.Checked):   velocityflag=True
        else:  velocityflag=False
        if(headflag==Qt.Checked):   headflag=True
        else:  headflag=False        

        if(flowflag==True) or (velocityflag==True) or (headflag==True):
            if(link_indx!=-1):#if the link was found
                %matplotlib qt 
                figure_tmp1,ax1 = plt.subplots() 
                if(flowflag):
                    plt.plot(link_flow,color='green',label=str("Link "+str(selected_link_name)+" flow"))      
                if(velocityflag):
                    plt.plot(link_velocity,color='blue',label=str("Link "+str(selected_link_name)+" velocity"))    
                if(headflag):
                    plt.plot(link_overhead,color='red',label=str("Link "+str(selected_link_name)+" overhead"))          
                plt.legend()
                plt.tight_layout()                    
                plt.show()
                %matplotlib inline                     
                time.sleep(0.5)
                
    def EpyT_showsimulationmapfortimnestamp_click(self):
        if(self.EpyT_model==None):
            print("Run simulation first")
            return
        timestamp=int(self.EpyT_timestamp_values_text_box_5.text())
        #this is how results are packed -  self.EpyT_results = [T,P,D,H,F,V,LH]
        if(timestamp>len(self.EpyT_results[0])):
            print("Time step is out of simulation result time scale")
            print("provide value up to "+str(len(self.EpyT_results[0])))
            return
        parameter_to_show=self.EpyTshowmapfortimestampcombo.currentText()
                
        if(parameter_to_show=="Node pressure"):
            Param=self.EpyT_results[1]#node pressure
        if(parameter_to_show=="Node demand"):
            Param=self.EpyT_results[2]#node demand
        if(parameter_to_show=="Node head"):
            Param=self.EpyT_results[3]#node head
        #if(parameter_to_show=="Node input"):
        #    Param=self.EpyT_results[4]#node intake
        Param=np.asarray(Param)
        Param=Param[timestamp,:]
        %matplotlib qt 
        figure_tmp,ax = plt.subplots() 
        ax = self.EpyT_model.plot(node_values = Param)        
        plt.tight_layout()                    
        plt.show()
        %matplotlib inline                     
        time.sleep(0.5)
    
    def EpyT_simulator_click(self):

        """
        if(self.Inp_Model_Path is None) or (): 
            print("Load .inp model first and repeat...") 
            return

        sim_steps_num=int(self.SimSteps_text_box.text())        
        terrain_max_h=float(np.max(np.asarray(self.Junc_elevation))-np.min(np.asarray(self.Junc_elevation)))
        max_rain_v=float(self.WNTR_MaxRain_values_text_box_4.text())
        waterIntakeFlag=self.WNTR_waterintakeflag.checkState()        
        waterIntakeScenario=self.comboBox.currentText()
        savealldata=self.Savealldatacheckbox.checkState()
        path_save_all_data=self.WNTR_SimSave_text_box_2.text() 
        clouds_gen_methods=self.CloudsGenMethods_comboBox_3.currentText()
        if(savealldata==Qt.Checked):   savealldata=True
        else:  savealldata=False
        if(waterIntakeFlag==Qt.Checked): waterIntakeFlag=True
        else:  waterIntakeFlag=False

        if(savealldata==True):
            now = datetime.now()
            dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
            path_save_all_data=path_save_all_data+"\\Exp_"+dt_string
            if not os.path.exists(path_save_all_data):
                os.makedirs(path_save_all_data)
        """
        
        self.ReadGlobalSimuationSettings()
        
        time_series=GenerateSpatialDomain(scenario=self.clouds_terrain_gen_scenario,                                                      
                                          nodes=self.Node_Names,
                                          time_series_length=self.sim_steps_num,
                                          nodes_X=self.Nodes_coord_X,
                                          nodes_Y=self.Nodes_coord_Y,
                                          spatial_domain_nRows=self.nRow,
                                          spatial_domain_nCol=self.nCol,
                                          rand_gen_type=self.clouds_gen_methods,#"const","nlmpy", "lorenz_attractor", "white_noise","const"
                                          clouds_max_h=self.max_rain_v,                                             
                                          terrain_max_h=self.terrain_max_h,
                                          clouds_grad_transition=6,
                                          save_history=self.savealldata,
                                          histry_dir=self.path_save_all_data,
                                          #this are the subdir names
                                          sub_dir_clouds_history=self.Sim_ub_dir_clouds_history,
                                          sub_dir_terrain_hist=self.Sim_sub_dir_terrain_hist,
                                          sub_dir_time_series_nodes_intake=self.Sim_sub_dir_time_series_nodes_intake,
                                          sub_dir_nodes_params=self.Sim_ub_dir_nodes_params
                                          )
        
        #create time series
        network,nodes_names,pipe_names=EpyT_makeNet(self.Graph,
                                                    NodeNames=self.Node_Names,
                                                    useWaterInputs=self.waterIntakeFlag,
                                                    PatternsTimeSerries=time_series)
        
        #R = network.getComputedTimeSeries_ENepanet() #network.getComputedTimeSeries()
        #H = network.getComputedHydraulicTimeSeries()
        #Q = network.getComputedQualityTimeSeries()
        #how to save - https://epanet-python-toolkit-epyt.readthedocs.io/en/latest/examples.html
        
        #h_time= H.Time
        #h_pressure=H.Pressure
        #h_velocity=H.Velocity

        network.openHydraulicAnalysis()
        network.initializeHydraulicAnalysis()
        tstep,P,D,H,F,V,LH,T= 1,[],[],[],[],[],[],[]
        cnt=1
        while True:
           t = network.runHydraulicAnalysis()           
           #nodes
           P.append(network.getNodePressure())
           D.append(network.getNodeActualDemand())
           H.append(network.getNodeHydraulicHead())            
           #links
           F.append(network.getLinkFlows())
           V.append(network.getLinkVelocity())
           LH.append(network.getLinkHeadloss())
            
           T.append(t)
           tstep=network.nextHydraulicAnalysisStep()           
           if(cnt>=self.sim_steps_num):
               break
           cnt=cnt+1
        network.closeHydraulicAnalysis()
        #hr = 0
        #d.plot(node_values = P[hr])
        self.EpyT_model   = network
        self.EpyT_results = [T,P,D,H,F,V,LH]
        self.EpyT_Nodes_Inputs=time_series
        self.EpyT_Nodes_Names_Inputs=self.Node_Names
        print("Simulation complete")
        print("Size of node pressure sim.data: "+str(np.shape(np.asarray(P))))
        print("Size of node pressure sim.data: "+str(np.shape(np.asarray(D))))
        print("Size of node pressure sim.data: "+str(np.shape(np.asarray(H))))
        print("Size of links flow sim.data: "+str(np.shape(np.asarray(F))))
        print("")
        #update combobox
        self.EpyT_node_list_comboBox_3.clear()
        self.EpyT_link_list_comboBox_4.clear()               
        nodes_n=self.EpyT_model.getNodeNameID()
        for t in nodes_n:
            self.EpyT_node_list_comboBox_3.addItem(str(t))        
        links_n=self.EpyT_model.getNodesConnectingLinksID()
        for t in links_n:
            self.EpyT_link_list_comboBox_4.addItem(str(t))
        #save data
        if(self.savealldata==True):            
            path_for_nodes_params=path_save_all_data+"\\"+self.Sim_ub_dir_nodes_params
            if not os.path.exists(path_for_nodes_params):
                os.makedirs(path_for_nodes_params)
            #nodes pressure
            subdir_Nodes_pressure=path_for_nodes_params+"\\NODES_PRESSURE"
            subdir_Nodes_DEMAND=path_for_nodes_params+"\\NODES_DEMAND"
            subdir_Nodes_OVERHEAD=path_for_nodes_params+"\\NODES_OVERHEAD"
            
            if not os.path.exists(subdir_Nodes_pressure):
                os.makedirs(subdir_Nodes_pressure)
            if not os.path.exists(subdir_Nodes_DEMAND):
                os.makedirs(subdir_Nodes_DEMAND)
            if not os.path.exists(subdir_Nodes_OVERHEAD):
                os.makedirs(subdir_Nodes_OVERHEAD)  
                
            for i in range(0,len(nodes_n)):
                cur_file_name=subdir_Nodes_pressure+"\\node_"+str(nodes_n[i])+"_pressure.csv"
                df = pd.DataFrame(np.asarray(P)[:,i])                    
                df.to_csv(cur_file_name, header=False, index=False, sep=' ')     
                cur_file_name_demand=subdir_Nodes_DEMAND+"\\node_"+str(nodes_n[i])+"_demand.csv"
                df = pd.DataFrame(np.asarray(D)[:,i])                    
                df.to_csv(cur_file_name_demand, header=False, index=False, sep=' ')    
                cur_file_name_overhead=subdir_Nodes_OVERHEAD+"\\node_"+str(nodes_n[i])+"_overhead.csv"
                df = pd.DataFrame(np.asarray(H)[:,i])                    
                df.to_csv(cur_file_name_overhead, header=False, index=False, sep=' ')    
                
            #link flows
            subdir_links_flows=path_for_nodes_params+"\\LINK_FLOWS"
            subdir_links_velocity=path_for_nodes_params+"\\LINK_VELOCITY"
            subdir_links_headloss=path_for_nodes_params+"\\LINK_HEADLOSS"
            if not os.path.exists(subdir_links_flows):
                os.makedirs(subdir_links_flows)
            if not os.path.exists(subdir_links_velocity):
                os.makedirs(subdir_links_velocity)
            if not os.path.exists(subdir_links_headloss):
                os.makedirs(subdir_links_headloss)
                
            for i in range(0,len(links_n)):
                cur_file_name=subdir_links_flows+"\\link_"+str(links_n[i])+"_flow.csv"
                df = pd.DataFrame(np.asarray(F)[:,i])                    
                df.to_csv(cur_file_name, header=False, index=False, sep=' ')     

                cur_file_name=subdir_links_velocity+"\\link_"+str(links_n[i])+"_velocity.csv"
                df1 = pd.DataFrame(np.asarray(V)[:,i])                    
                df1.to_csv(cur_file_name, header=False, index=False, sep=' ')   

                cur_file_name=subdir_links_velocity+"\\link_"+str(links_n[i])+"_headloss.csv"
                df2 = pd.DataFrame(np.asarray(HL)[:,i])                    
                df2.to_csv(cur_file_name, header=False, index=False, sep=' ')   
            
    def EpyT_showmodel_click(self):
        
        if(self.Inp_Model_Path is None) or (): 
            print("Load .inp model first and repeat...") 
            return
            
        sim_steps_num=1
        terrain_max_h=float(np.max(np.asarray(self.Junc_elevation))-np.min(np.asarray(self.Junc_elevation)))
        max_rain_v=float(self.WNTR_MaxRain_values_text_box_4.text())
        #waterIntakeFlag=self.WNTR_waterintakeflag.checkState()        
        #waterIntakeScenario=self.comboBox.currentText()
        #savealldata=self.Savealldatacheckbox.checkState()
        #path_save_all_data=self.WNTR_SimSave_text_box_2.text()        
        #if(savealldata==Qt.Checked):   savealldata=True
        #else:  savealldata=False
        #if(waterIntakeFlag==Qt.Checked): waterIntakeFlag=True
        #else:  waterIntakeFlag=False
                
        time_series=GenerateSpatialDomain(scenario="Clouds",nodes=self.Node_Names,
                                                     time_series_length=sim_steps_num,
                                                     nodes_X=self.Nodes_coord_X,
                                                     nodes_Y=self.Nodes_coord_Y,
                                                     spatial_domain_nRows=self.nRow,
                                                     spatial_domain_nCol=self.nCol,
                                                     rand_gen_type="nlmpy",#"const","nlmpy", "lorenz_attractor", "white_noise","const"
                                                     clouds_max_h=max_rain_v,                                             
                                                     terrain_max_h=terrain_max_h,
                                                     clouds_grad_transition=6,
                                                     save_history=False,
                                                     histry_dir="",
                                                     #this are the subdir names
                                                     sub_dir_clouds_history=self.Sim_ub_dir_clouds_history,
                                                     sub_dir_terrain_hist=self.Sim_sub_dir_terrain_hist,
                                                     sub_dir_time_series_nodes_intake=self.Sim_sub_dir_time_series_nodes_intake,
                                                     sub_dir_nodes_params=self.Sim_ub_dir_nodes_params
                                                 )
        
        #create time series
        network,nodes_names,pipe_names=EpyT_makeNet(self.Graph,NodeNames=self.Node_Names,useWaterInputs=False,PatternsTimeSerries=time_series)
        %matplotlib qt 
        figure_tmp,ax = plt.subplots() 
        ax = network.plot()
        #figure_tmp.suptitle(bs, fontsize=16)
        title=str("EpyT model from NetworkX graph")
        ax.set_title(title)
        plt.tight_layout()                    
        plt.show()
        %matplotlib inline                     
        time.sleep(0.5)

    def EpyT_showmodelelevation_click(self):
        if(self.Inp_Model_Path is None) or (): 
            print("Load .inp model first and repeat...") 
            return

        #sim_steps_num=int(self.SimSteps_text_box.text())
        sim_steps_num=1
        terrain_max_h=float(np.max(np.asarray(self.Junc_elevation))-np.min(np.asarray(self.Junc_elevation)))
        max_rain_v=float(self.WNTR_MaxRain_values_text_box_4.text())
                       
        time_series=GenerateSpatialDomain(scenario="Clouds",nodes=self.Node_Names,
                                                     time_series_length=sim_steps_num,
                                                     nodes_X=self.Nodes_coord_X,
                                                     nodes_Y=self.Nodes_coord_Y,
                                                     spatial_domain_nRows=self.nRow,
                                                     spatial_domain_nCol=self.nCol,
                                                     rand_gen_type="nlmpy",#"const","nlmpy", "lorenz_attractor", "white_noise","const"
                                                     clouds_max_h=max_rain_v,                                             
                                                     terrain_max_h=terrain_max_h,
                                                     clouds_grad_transition=6,
                                                     save_history=False,
                                                     histry_dir="",
                                                     #this are the subdir names
                                                     sub_dir_clouds_history=self.Sim_ub_dir_clouds_history,
                                                     sub_dir_terrain_hist=self.Sim_sub_dir_terrain_hist,
                                                     sub_dir_time_series_nodes_intake=self.Sim_sub_dir_time_series_nodes_intake,
                                                     sub_dir_nodes_params=self.Sim_ub_dir_nodes_params
                                                 )
        #check api - https://epanet-python-toolkit-epyt.readthedocs.io/en/latest/api.html
        #check api - 
        #create time series
        network,nodes_names,pipe_names=EpyT_makeNet(self.Graph,NodeNames=self.Node_Names,useWaterInputs=False,PatternsTimeSerries=time_series)
        %matplotlib qt 
        figure_tmp,ax = plt.subplots() 
        #d.getNodePressure()
        #elevations=[]
        #for i in range(0,):
        #    value=d.getNodeElevations(index_node)
        #    elevations.append(value)
        ax = network.plot(title="EpyT model elevation",node_values = network.getNodeElevations(),min_colorbar=np.min(np.asarray(self.Junc_elevation)), max_colorbar=np.max(np.asarray(self.Junc_elevation)))
        #figure_tmp.suptitle(bs, fontsize=16)
        plt.tight_layout()                    
        plt.show()
        %matplotlib inline                     
        time.sleep(0.5)
        

app = QtWidgets.QApplication(sys.argv)
window = MainWindow()
window.show()
sys.exit(app.exec())


-----------------------------------
---------*.inp File opened---------
-----------------------------------

File name: ['C:/Users/shse/Documents/ANACONDA/UTwin_notebooks/UTWIN_notebooks/faf_feb16_apr20.inp']
Nodes num.: 440
Counduits num.: 431
Junctions num.:427
Storages num.:5
Pumps num.:5
Outfalls num.:8

-----------------------------------
-----------------------------------
Map dimensions:
X min: 697526.013
X max: 699814.894
Y min: 248338.455
Y max: 249842.635

Nodes number: 440
Simulation time step 0.05243 s
Total Time Step in this simulation 19


C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\initialize.py:180: RuntimeWarning: invalid value encountered in sqrt
  start_demand_coeff = demand[0]/ np.sqrt(Hs)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\initialize.py:185: RuntimeWarning: invalid value encountered in sqrt
  end_demand_coeff = demand[1] / np.sqrt(He)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\initialize.py:136: RuntimeWarning: invalid value encountered in sqrt
  pipe.start_node.emitter_discharge[0] = pipe.start_node.emitter_coeff * np.sqrt(Hsa)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\initialize.py:137: RuntimeWarning: invalid value encountered in sqrt
  pipe.start_node.demand_discharge[0] = pipe.start_node.demand_coeff * np.sqrt(Hsa)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\initialize.py:139: RuntimeWarning: invalid value encountered in sqrt
  pipe.end_node.emitter_discharge[0] = pipe.end_node.emitter_coeff * np.sqrt(Hea)
C:\ProgramDa

Simultion progress(s): 0 out of 5000
Transient simulation completed 5 %...
Initial condition discrepancy of pressure (-200.0000 m) on the Notentlastung_48b node
Initial condition discrepancy of pressure (-200.0000 m) on the Bacheinlauf30 node
Initial condition discrepancy of pressure (-200.0000 m) on the Node_8 node
Transient simulation completed 10 %...


C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\main.py:313: UserWarning: Negative pressure on node 603alt_SBW.                        Backflow stopped by reverse flow preventer.
  warnings.warn("Negative pressure on node %s.\
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\main.py:326: UserWarning: Negative pressure on node 603alt_SBW.                            Backflow stopped by reverse flow preventer.
  warnings.warn("Negative pressure on node %s.\
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\main.py:216: UserWarning: Negative pressure on node 604. Backflow stopped by reverse flow preventer.
  warnings.warn("Negative pressure on node %s. Backflow stopped by reverse flow preventer." %pipe.start_node.name)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\main.py:226: UserWarning: Negative pressure on node 604 Backflow stopped by reverse flow preventer.
  warnings.warn("Negative pressure on node %s Backflow stopped by reverse flow pre

Estimated simulation time 0:00:01.994430
Transient simulation completed 15 %...
Transient simulation completed 21 %...
Transient simulation completed 26 %...
Transient simulation completed 31 %...
Transient simulation completed 36 %...
Transient simulation completed 42 %...


C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:409: RuntimeWarning: overflow encountered in scalar multiply
  J2 = f*dt/2./D*V2*abs(V2)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:405: RuntimeWarning: overflow encountered in scalar multiply
  J1 = f*dt/2./D*V1*abs(V1)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:413: RuntimeWarning: invalid value encountered in scalar add
  HP[i] = (C[0,0] + C[1,0]) / (C[0,1] + C[1,1])
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:131: RuntimeWarning: overflow encountered in scalar multiply
  Js = f*dt/2./D*V*abs(V) #steady friction
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:410: RuntimeWarning: invalid value encountered in scalar add
  C[1,0] = -V2+ ga*H2 + J2 + ga* dt *V2*theta
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:410: RuntimeWarning: invalid value encountered in scalar multiply
  C[1,0] = -V2+ 

Transient simulation completed 47 %...
Transient simulation completed 52 %...
Transient simulation completed 57 %...
Transient simulation completed 63 %...
Transient simulation completed 68 %...
Transient simulation completed 73 %...
Transient simulation completed 78 %...
Transient simulation completed 84 %...
Transient simulation completed 89 %...
Transient simulation completed 94 %...
Simultion progress(s): 1 out of 5000
Transient simulation completed 5 %...
Initial condition discrepancy of pressure (-200.0000 m) on the Notentlastung_48b node
Initial condition discrepancy of pressure (-200.0000 m) on the Bacheinlauf30 node
Initial condition discrepancy of pressure (-200.0000 m) on the Node_8 node
Transient simulation completed 10 %...
Estimated simulation time 0:00:01.875642
Transient simulation completed 15 %...
Transient simulation completed 21 %...
Transient simulation completed 26 %...
Transient simulation completed 31 %...
Transient simulation completed 36 %...
Transient simulat

In [55]:
#TSNet
fn="C:\\Users\\shse\\Documents\\ANACONDA\\UTwin_notebooks\\UTWIN_notebooks\\faf_feb16_apr20.inp"
L1,L2,L3,L4,L5,L6,L7,L8,L8_1,L8_2,L8_3,L8_4,L8_5,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21,L22,L23,L24,L25=utr.OpenInp(fn)
Graph,points,pos2=MakeGraph(L1,L2,L3,L4,L5,L6,L7,L8,L8_1,L8_2,L8_3,L8_4,L8_5,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21,L22,L23,L24,L25)
model_wntr,n_names,l_names = WNTR_show_elevation_for_model(G=Graph,show_elevation=False,show_population=False)
model_path="C:\\Tmp\\hhhj.inp"
wntr.network.io.write_inpfile (model_wntr,model_path)
#os.remove(model_path)
tm = tsnet.network.TransientModel(model_path)#'C:\\Users\\shse\\Documents\\ANACONDA\\UTwin_notebooks\\UTWIN_notebooks\\EpyT_init_model.inp')

nodes_list= tm.node_name_list
pipes_list= tm.link_name_list

sim_steps_num=2

results_nodes_head=[]
results_nodes_emit_discharge=[]
results_nodes_demand_discharge=[]

for l in nodes_list:
    results_nodes_head.append(np.empty)
    results_nodes_emit_discharge.append(np.empty)
    results_nodes_demand_discharge.append(np.empty)
    
results_start_head=[]
results_end_head=[]
results_start_velocity=[]
results_end_velocity=[]
results_start_flowrate=[]
results_end_flowrate=[]

for l in pipes_list:
    results_start_head.append(np.empty)
    results_end_head.append(np.empty)
    results_start_velocity.append(np.empty)
    results_end_velocity.append(np.empty)
    results_start_flowrate.append(np.empty)
    results_end_flowrate.append(np.empty)

#we go one sec by another
for i in range(0,sim_steps_num):     

    print("Simultion progress(s): "+str(i))
    
    Wave_Speed=12 #m/s
    t0=0
    tf = 1   
    tm.set_wavespeed(Wave_Speed) # m/s
    tm.set_time(tf)

    for l in nodes_list:
        junction = tm.get_node(l)
        junction.level=9
    
    tm = tsnet.simulation.Initializer(tm,t0)
    tm = tsnet.simulation.MOCSimulator(tm)

    #results nodes
    for l in nodes_list:
        
        node=tm.get_node(l)
        indx=nodes_list.index(l)
        head = node.head
        emitter_discharge = node.emitter_discharge
        demand_discharge = node.demand_discharge     

        tmp1=np.asarray(results_nodes_head[indx])
        tmp2=np.concatenate((tmp1, np.asarray(head)), axis=None)
        results_nodes_head[indx]=np.empty
        results_nodes_head[indx]=tmp2

        tmp1=np.asarray(results_nodes_emit_discharge[indx])
        tmp2=np.concatenate((tmp1, np.asarray(emitter_discharge)), axis=None)
        results_nodes_emit_discharge[indx]=np.empty
        results_nodes_emit_discharge[indx]=tmp2

        tmp1=np.asarray(results_nodes_demand_discharge[indx])
        tmp2=np.concatenate((tmp1, np.asarray(demand_discharge)), axis=None)
        results_nodes_demand_discharge[indx]=np.empty
        results_nodes_demand_discharge[indx]=tmp2
        
        #results links
    for l in pipes_list:
        
        pipe=tm.get_link(l)
        indx=pipes_list.index(l)
        
        start_head = pipe.start_node_head
        end_head = pipe.end_node_head
        start_velocity = pipe.start_node_velocity
        end_velocity = pipe.end_node_velocity
        start_flowrate = pipe.start_node_flowrate
        end_flowrate = pipe.end_node_flowrate

  
        tmp1=np.asarray(results_start_head[indx])
        tmp2=np.concatenate((tmp1, np.asarray(start_head)), axis=None)
        results_start_head[indx]=np.empty
        results_start_head[indx]=tmp2

        tmp1=np.asarray(results_end_head[indx])
        tmp2=np.concatenate((tmp1, np.asarray(end_head)), axis=None)
        results_end_head[indx]=np.empty
        results_end_head[indx]=tmp2
        
        tmp1=np.asarray(results_start_velocity[indx])
        tmp2=np.concatenate((tmp1, np.asarray(start_velocity)), axis=None)
        results_start_velocity[indx]=np.empty
        results_start_velocity[indx]=tmp2
        
        tmp1=np.asarray(results_end_velocity[indx])
        tmp2=np.concatenate((tmp1, np.asarray(end_velocity)), axis=None)
        results_end_velocity[indx]=np.empty
        results_end_velocity[indx]=tmp2

        tmp1=np.asarray(results_start_flowrate[indx])
        tmp2=np.concatenate((tmp1, np.asarray(start_flowrate)), axis=None)
        results_start_flowrate[indx]=np.empty
        results_start_flowrate[indx]=tmp2

        tmp1=np.asarray(results_end_flowrate[indx])
        tmp2=np.concatenate((tmp1, np.asarray(end_flowrate)), axis=None)
        results_end_flowrate[indx]=np.empty
        results_end_flowrate[indx]=tmp2
                
    #tm.plot_node_head(L1[0])
    print("")
    print("___________________________")
    print("TSnet - simulation complete:")
    print("Simulation results shape for nodes: " + str(np.shape(np.asarray(results_start_head))))
    print("Simulation results shape for links: " + str(np.shape(np.asarray(results_start_velocity))))
    print("___________________________")
    print("")

Map dimensions:
X min: 697526.013
X max: 699814.894
Y min: 248338.455
Y max: 249842.635

Nodes number: 440
Simultion progress(s): 0
Simulation time step 0.07864 s
Total Time Step in this simulation 12


C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\initialize.py:180: RuntimeWarning: invalid value encountered in sqrt
  start_demand_coeff = demand[0]/ np.sqrt(Hs)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\initialize.py:185: RuntimeWarning: invalid value encountered in sqrt
  end_demand_coeff = demand[1] / np.sqrt(He)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\initialize.py:136: RuntimeWarning: invalid value encountered in sqrt
  pipe.start_node.emitter_discharge[0] = pipe.start_node.emitter_coeff * np.sqrt(Hsa)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\initialize.py:137: RuntimeWarning: invalid value encountered in sqrt
  pipe.start_node.demand_discharge[0] = pipe.start_node.demand_coeff * np.sqrt(Hsa)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\initialize.py:139: RuntimeWarning: invalid value encountered in sqrt
  pipe.end_node.emitter_discharge[0] = pipe.end_node.emitter_coeff * np.sqrt(Hea)
C:\ProgramDa

Transient simulation completed 8 %...


C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\main.py:216: UserWarning: Negative pressure on node 591j. Backflow stopped by reverse flow preventer.
  warnings.warn("Negative pressure on node %s. Backflow stopped by reverse flow preventer." %pipe.start_node.name)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\main.py:226: UserWarning: Negative pressure on node 591j Backflow stopped by reverse flow preventer.
  warnings.warn("Negative pressure on node %s Backflow stopped by reverse flow preventer." %pipe.start_node.name)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\main.py:313: UserWarning: Negative pressure on node 591m.                        Backflow stopped by reverse flow preventer.
  warnings.warn("Negative pressure on node %s.\
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\main.py:326: UserWarning: Negative pressure on node 591m.                            Backflow stopped by reverse flow preventer.
  warnings.warn("Negative 

Initial condition discrepancy of pressure (-200.0000 m) on the Notentlastung_48b node
Initial condition discrepancy of pressure (-200.0000 m) on the Bacheinlauf30 node
Initial condition discrepancy of pressure (-200.0000 m) on the Node_8 node
Transient simulation completed 16 %...
Estimated simulation time 0:00:02.542236
Transient simulation completed 25 %...
Transient simulation completed 33 %...
Transient simulation completed 41 %...
Transient simulation completed 50 %...
Transient simulation completed 58 %...


...

Transient simulation completed 8 %...


C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:1013: UserWarning: Error: The quadratic equation has no real solution (leakage)
  warnings.warn('Error: The quadratic equation has no real solution (leakage)')
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\main.py:216: UserWarning: Negative pressure on node 101. Backflow stopped by reverse flow preventer.
  warnings.warn("Negative pressure on node %s. Backflow stopped by reverse flow preventer." %pipe.start_node.name)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\main.py:226: UserWarning: Negative pressure on node 101 Backflow stopped by reverse flow preventer.
  warnings.warn("Negative pressure on node %s Backflow stopped by reverse flow preventer." %pipe.start_node.name)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\main.py:216: UserWarning: Negative pressure on node 102a. Backflow stopped by reverse flow preventer.
  warnings.warn("Negative pressure on node %s. Backflow st

Initial condition discrepancy of pressure (-200.0000 m) on the Notentlastung_48b node
Initial condition discrepancy of pressure (-200.0000 m) on the Bacheinlauf30 node
Initial condition discrepancy of pressure (-200.0000 m) on the Node_8 node
Transient simulation completed 16 %...
Estimated simulation time 0:00:02.921544
Transient simulation completed 25 %...
Transient simulation completed 33 %...
Transient simulation completed 41 %...
Transient simulation completed 50 %...
Transient simulation completed 58 %...
Transient simulation completed 66 %...
Transient simulation completed 75 %...


C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:409: RuntimeWarning: overflow encountered in scalar multiply
  J2 = f*dt/2./D*V2*abs(V2)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:405: RuntimeWarning: overflow encountered in scalar multiply
  J1 = f*dt/2./D*V1*abs(V1)
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:413: RuntimeWarning: invalid value encountered in scalar add
  HP[i] = (C[0,0] + C[1,0]) / (C[0,1] + C[1,1])
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:131: RuntimeWarning: overflow encountered in scalar multiply
  Js = f*dt/2./D*V*abs(V) #steady friction
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:410: RuntimeWarning: invalid value encountered in scalar add
  C[1,0] = -V2+ ga*H2 + J2 + ga* dt *V2*theta
C:\ProgramData\anaconda3\Lib\site-packages\tsnet\simulation\solver.py:410: RuntimeWarning: invalid value encountered in scalar multiply
  C[1,0] = -V2+ 

Transient simulation completed 83 %...
Transient simulation completed 91 %...

___________________________
TSnet - simulation complete:
Simulation results shape for nodes: (431, 25)
Simulation results shape for links: (431, 25)
___________________________



In [ ]:
import pyswmm as swmm
from pyswmm import Simulation, Nodes, Links
fn="C:\\Users\\shse\\Documents\\ANACONDA\\UTwin_notebooks\\UTWIN_notebooks\\empty_model.inp"
sim=Simulation(fn)
nodes=swmm.Nodes(sim)
links=swmm.Links(sim)
print(len(list(nodes)))
print(len(list(links)))
sim.step_advance(1)
for step in sim:
        print(sim.current_time)

In [ ]:
from epyt import epanet
import os
import numpy as np

current_working_directory = os.getcwd()
inp_init_path=current_working_directory+"\\EpyT_init_model.inp"
d = epanet(inp_init_path)
link_count = d.getLinkCount()
nodes_n=d.getNodeNameID()
links_n=[]
for i in range(0,link_count):
    try:
        cur_name=d.getLinkNameID(i)
        links_n.append(cur_name)
    except:
        pass

print(links_n)

In [ ]:
nodes_list=list(self.EpyT_model.getNodeNameID()       )
        links_list=list(self.EpyT_model.getNodesConnectingLinksID())

In [ ]:
from epyt import epanet
import os
import numpy as np

current_working_directory = os.getcwd()
inp_init_path=current_working_directory+"\\EpyT_init_model.inp"
d = epanet(inp_init_path)
d.openHydraulicAnalysis()
d.initializeHydraulicAnalysis()
tstep, P = 1, []
cnt=0
while True:
   t = d.runHydraulicAnalysis()
   P.append(d.getNodePressure())
   tstep=d.nextHydraulicAnalysisStep()
   #print(np.shape(np.asarray(P)))
   if(cnt>=20):
       break
   cnt=cnt+1
d.closeHydraulicAnalysis()
hr = 0
d.plot(node_values = P[hr])

In [ ]:
#check https://github.com/SWIL-IITK/EPyT-C
#https://github.com/WaterFutures/EPyT-Flow

In [ ]:
from epyt import epanet
import os

current_working_directory = os.getcwd()
inp_init_path=current_working_directory+"\\EpyT_init_model.inp"
d = epanet(inp_init_path)
link_count = d.getLinkCount()
nodes_count= d.getNodeCount()
rules_ID = d.getRules()
control_count=d.getControlRulesCount()
print("Initialization of EpyT model:")
print(" - number of nodes: "+str(nodes_count))
print(" - number of links: "+str(link_count))
print(" - number of rules: "+str(len(rules_ID)))
print(" - number of controls: "+str(control_count))

print("")
nodes_n=d.getNodeNameID()
links_n=[]
for i in range(0,link_count):
    try:
        cur_name=d.getLinkNameID(i)
        links_n.append(cur_name)
    except:
        pass

for m in links_n:
    d.deleteLink(m)
for k in nodes_n:
    d.deleteNode(k)
for v in range(0,control_count):
    d.deleteControls()
d.deleteRules()

print("Cleaned EpyT model:")
print(" - number of nodes: "+str(d.getLinkCount()))
print(" - number of links: "+str(d.getNodeCount()))
print(" - number of rules: "+str(len(d.getRules())))
print(" - number of controls: "+str(d.getControlRulesCount()))
print("")
print("Initialize networkX model...")
print("")


In [ ]:
def EpyT_makeNet(graph=None, useWaterInputs=True,NodeNames=[],PatternsTimeSerries=[]):
    
    #https://github.com/KIOS-Research/EPyT/blob/main/epyt/examples/EX1_Plot_network_topology.ipynb    
    #https://pypi.org/project/epyt/
    G=graph    
    if(graph==None):
        return None, None, None
    #load model that will be earsed and new model will be introduced on top
    current_working_directory = os.getcwd()
    inp_init_path=current_working_directory+"\\EpyT_init_model.inp"
    d = epanet(inp_init_path)
    link_count = d.getLinkCount()
    nodes_count= d.getNodeCount()
    rules_ID = d.getRules()
    control_count=d.getControlRulesCount()
    print("Initialization of EpyT model:")
    print(" - number of nodes: "+str(nodes_count))
    print(" - number of links: "+str(link_count))
    print(" - number of rules: "+str(len(rules_ID)))
    print(" - number of controls: "+str(control_count))
    
    print("")
    nodes_n=d.getNodeNameID()
    links_n=[]
    for i in range(0,link_count):
        try:
            cur_name=d.getLinkNameID(i)
            links_n.append(cur_name)
        except:
            pass
    
    for m in links_n:
        d.deleteLink(m)
    for k in nodes_n:
        d.deleteNode(k)
    for v in range(0,control_count):
        d.deleteControls()
    d.deleteRules()
    
    print("Cleaned EpyT model:")
    print(" - number of nodes: "+str(d.getLinkCount()))
    print(" - number of links: "+str(d.getNodeCount()))
    print(" - number of rules: "+str(len(d.getRules())))
    print(" - number of controls: "+str(d.getControlRulesCount()))
    print("")
    print("Initialize networkX model...")
    print("")
        
    Used_Nodes_Names=[]
    Pipes_Names=[]
    
    for edge in  G.edges(data=True): #u,outer_d in G.nodes(data=True):      
        
        n1=edge[0]
        n2=edge[1]           
      
        if((n1 in Used_Nodes_Names)==False):                                             
            if(int(G.nodes[n1]["is_junction"])==True):     
                #https://epanet-python-toolkit-epyt.readthedocs.io/en/latest/api.html
                x_coord=float(G.nodes[n1]["X"])
                y_coord=float(G.nodes[n1]["Y"])
                coordinates=[x_coord,y_coord]
                junctionElevation = float(G.nodes[n1]["junction_elevation"])
                demand = 50                               
                if (useWaterInputs==True) and (n1 in NodeNames)==True:
                    index=NodeNames.index(n1)
                    patern_name=str('pat_'+n1)
                    time_series=PatternsTimeSerries[index]                    
                    d.addPattern(patern_name,time_series)                         
                    #ptrn=wn.getPatternNameID(patern_name)                   
                    d.addNodeJunction(n1,coordinates,junctionElevation,demand,patern_name)             
                else:                    
                    d.addNodeJunction(n1,coordinates,junctionElevation,demand)   
                                
            if(int(G.nodes[n1]["isStorage"])==True or int(G.nodes[n1]["isOutfall"])==True):                 
                x_coord=float(G.nodes[n2]["X"])
                y_coord=float(G.nodes[n2]["Y"])
                reservoirCoords = [x_coord, y_coord]
                reservoirIndex = d.addNodeReservoir(n1, reservoirCoords)
                
            Used_Nodes_Names.append(n1)    
           
        # NODE 2     
       
        if((n2 in Used_Nodes_Names)==False):               
            if(int(G.nodes[n2]["is_junction"])==True):         

                #https://epanet-python-toolkit-epyt.readthedocs.io/en/latest/api.html
                x_coord=float(G.nodes[n2]["X"])
                y_coord=float(G.nodes[n2]["Y"])
                coordinates=[x_coord,y_coord]
                junctionElevation = float(G.nodes[n2]["junction_elevation"])
                demand = 50           
                                
                if (useWaterInputs==True) and (n2 in NodeNames)==True:
                    index=NodeNames.index(n2)
                    patern_name=str('pat_'+n2)
                    time_series=PatternsTimeSerries[index]                    
                    d.addPattern(patern_name,time_series)                         
                    d.addNodeJunction(n2,coordinates,junctionElevation,demand,patern_name)              
                else:                    
                    d.addNodeJunction(n2,coordinates,junctionElevation,demand)   
                    
            if(int(G.nodes[n2]["isStorage"])==True or int(G.nodes[n2]["isOutfall"])==True):                
                x_coord=float(G.nodes[n2]["X"])
                y_coord=float(G.nodes[n2]["Y"])
                reservoirCoords = [x_coord, y_coord]
                reservoirIndex = d.addNodeReservoir(n2, reservoirCoords)   
                
            Used_Nodes_Names.append(n2)       
        #https://epanet-python-toolkit-epyt.readthedocs.io/en/latest/api.html        
        pipeID=str(n1)+"_"+str(n2)
        length=float(G.get_edge_data(*edge)["length"])
        diameter=float(G.get_edge_data(*edge)["x_cross_section"])#['weight'])
        roughness=float(G.get_edge_data(*edge)['roughness'])        
        minorLossCoeff=0.0       
        #link_names = wn.getLinkNameID()
        #print(pipeID)
        #if(True):
        #    print(length)
        #    print(diameter)
        #    print(roughness)
        #    print(minorLossCoeff)           
        pipe=d.addLinkPipe(pipeID, n1, n2,length,diameter,roughness,minorLossCoeff)  
        Pipes_Names.append(pipeID)
        #print("pipe added")
        
    print("") 
    print("Transfered from NetworkX EpyT model:")
    print(" - number of nodes: "+str(d.getLinkCount()))
    print(" - number of links: "+str(d.getNodeCount()))
    print(" - number of rules: "+str(len(d.getRules())))
    print(" - number of controls: "+str(d.getControlRulesCount()))
    print("")    
        
    return d, Used_Nodes_Names, Pipes_Names

In [ ]:
fn="C:\\Users\\shse\\Documents\\ANACONDA\\UTwin_notebooks\\UTWIN_notebooks\\faf_feb16_apr20.inp"
L1,L2,L3,L4,L5,L6,L7,L8,L8_1,L8_2,L8_3,L8_4,L8_5,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21,L22,L23,L24,L25=utr.OpenInp(fn)
Graph,points,pos2=MakeGraph(L1,L2,L3,L4,L5,L6,L7,L8,L8_1,L8_2,L8_3,L8_4,L8_5,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21,L22,L23,L24,L25)

time_series=GenerateSpatialDomain(scenario="Clouds",nodes=L1,
                                                     time_series_length=5000,
                                                     nodes_X=L2,
                                                     nodes_Y=L3,
                                                     spatial_domain_nRows=50,
                                                     spatial_domain_nCol=50,
                                                     rand_gen_type="nlmpy",#"const","nlmpy", "lorenz_attractor", "white_noise","const"
                                                     clouds_max_h=0.76,                                             
                                                     terrain_max_h=0.76,
                                                     clouds_grad_transition=6,
                                                     save_history=False,
                                                     histry_dir="",
                                                     #this are the subdir names
                                                     sub_dir_clouds_history="",
                                                     sub_dir_terrain_hist="",
                                                     sub_dir_time_series_nodes_intake="",
                                                     sub_dir_nodes_params=""
                                                 )


print(len(time_series))
print(len(L1))
#create time series
network,nodes_names,pipe_names=EpyT_makeNet(Graph,NodeNames=L1,useWaterInputs=True,PatternsTimeSerries=time_series)
print("-----")
print("Nodes returned to simulator: "+str(len(nodes_names)))
print("Links returned to simulator:"+str(len(pipe_names)))
network.plot()

In [ ]:
def EpyT_makeNet(graph=None, path="",useWaterInputs=True,NodeNames=[],PatternsTimeSerries=[]):
    
    #https://github.com/KIOS-Research/EPyT/blob/main/epyt/examples/EX1_Plot_network_topology.ipynb    
    #https://pypi.org/project/epyt/
    G=graph
    if(path!=""):
        d = epanet(path)
        return d
    if(graph==None):
        return None
        
    wn = epanet()
    Used_Nodes_Names=[]
    Pipes_Names=[]
    
    for edge in  G.edges(data=True): #u,outer_d in G.nodes(data=True):      
        
        n1=edge[0]
        n2=edge[1]   
        
        print("")
        print("Nodes")
        print(n1)
        print(n2)
        print("")
                
        # NODE 1  
        print("")
        print("Node 1")
        
        if((n1 in Used_Nodes_Names)==False):                                             
            if(int(G.nodes[n1]["is_junction"])==True):     
                #https://epanet-python-toolkit-epyt.readthedocs.io/en/latest/api.html
                x_coord=float(G.nodes[n1]["X"])
                y_coord=float(G.nodes[n1]["Y"])
                coordinates=[x_coord,y_coord]
                junctionElevation = float(G.nodes[n1]["junction_elevation"])
                demand = 50           
                                
                if (useWaterInputs==True) and (n1 in NodeNames)==True:
                    index=NodeNames.index(n1)
                    patern_name=str('pat_'+n1)
                    time_series=PatternsTimeSerries[index]
                    print(index)
                    wn.addPattern(patern_name,time_series)                         
                    #ptrn=wn.getPatternNameID(patern_name)         
                    print("Added as Junction with time series")
                    wn.addNodeJunction(n1,coordinates,junctionElevation,demand,patern_name)             
                else:
                    print("Added as Junction without time series")
                    wn.addNodeJunction(n1,coordinates,junctionElevation,demand)   
                                
            if(int(G.nodes[n1]["isStorage"])==True or int(G.nodes[n1]["isOutfall"])==True):       
                print("Added as storage ")
                x_coord=float(G.nodes[n2]["X"])
                y_coord=float(G.nodes[n2]["Y"])
                reservoirCoords = [x_coord, y_coord]
                reservoirIndex = wn.addNodeReservoir(n1, reservoirCoords)
                
            Used_Nodes_Names.append(n1)    
           
        # NODE 2     
        print("")
        print("Node 2")
        
        if((n2 in Used_Nodes_Names)==False):               
            if(int(G.nodes[n2]["is_junction"])==True):         

                #https://epanet-python-toolkit-epyt.readthedocs.io/en/latest/api.html
                x_coord=float(G.nodes[n2]["X"])
                y_coord=float(G.nodes[n2]["Y"])
                coordinates=[x_coord,y_coord]
                junctionElevation = float(G.nodes[n2]["junction_elevation"])
                demand = 50           
                                
                if (useWaterInputs==True) and (n2 in NodeNames)==True:
                    index=NodeNames.index(n2)
                    patern_name=str('pat_'+n2)
                    time_series=PatternsTimeSerries[index]
                    print(index)
                    wn.addPattern(patern_name,time_series)                         
                    #ptrn=wn.getPatternNameID(patern_name) 
                    print("Added as Junction with time series")
                    wn.addNodeJunction(n2,coordinates,junctionElevation,demand,patern_name)              
                else:
                    print("Node added without time series")
                    wn.addNodeJunction(n2,coordinates,junctionElevation,demand)   
                    
            if(int(G.nodes[n2]["isStorage"])==True or int(G.nodes[n2]["isOutfall"])==True):
                print("Added as storage ")
                x_coord=float(G.nodes[n2]["X"])
                y_coord=float(G.nodes[n2]["Y"])
                reservoirCoords = [x_coord, y_coord]
                reservoirIndex = wn.addNodeReservoir(n2, reservoirCoords)   
                
            Used_Nodes_Names.append(n2)     
            

        print("---------------------")
        print("length of nodes used:"+str(len(Used_Nodes_Names)))
        print("---------------------")
        print("")
        print("Ading pipe")
        #https://epanet-python-toolkit-epyt.readthedocs.io/en/latest/api.html        
        pipeID=str(n1)+"-"+str(n2)
        length=float(G.get_edge_data(*edge)["length"])
        diameter=float(G.get_edge_data(*edge)["x_cross_section"])#['weight'])
        roughness=float(G.get_edge_data(*edge)['roughness'])        
        minorLossCoeff=0.0
        print("pipe params:")
        print(length)
        print(diameter)
        print(roughness)
        print(minorLossCoeff)
        #link_names = wn.getLinkNameID()
        wn.addLinkPipe(pipeID, n1, n2,length,diameter,roughness,minorLossCoeff)  
        Pipes_Names.append(pipeID)   
        print("Pipe added with ID:" +str(pipeID))
        
    return wn, Used_Nodes_Names, Pipes_Names

In [ ]:
def GenerateSpatialDomain1(scenario="Clouds",nodes=None,
                                             time_series_length=500,
                                             nodes_X=None,
                                             nodes_Y=None,
                                             spatial_domain_nRows=50,
                                             spatial_domain_nCol=50,
                                             spatial_domain_maxMapl=1.5,
                                             rand_gen_type="nlmpy",#"const","nlmpy", "lorenz_attractor", "white_noise","const"
                                             clouds_grad_transition=0,
                                             #save everything related to the simulation
                                             save_history=True,
                                             histry_dir="",
                                             #this are the subdir names
                                             sub_dir_clouds_history="Clouds",
                                             sub_dir_terrain_hist="Terrain",
                                             sub_dir_time_series_nodes_intake="Nodes_Intake",
                                             sub_dir_nodes_params="Nodes_params"
                          ):

    
    
    #variables
    nRow=spatial_domain_nRows
    nCol=spatial_domain_nCol
    h=spatial_domain_maxMapl
    
    #prepare the time series and return variables
    #modes inputs
    rain_node_input=[]
    for i in range(0,len(nodes)):
        rain_node_input.append([])
    #this is terrain
    terrain_map=None
    watershed_map=None
        
    #prepare the coordinates convertions
    min_x_coord= np.min(np.asarray(nodes_X))
    max_x_coord= np.max(np.asarray(nodes_X))
    min_y_coord= np.min(np.asarray(nodes_Y))
    max_y_coord= np.max(np.asarray(nodes_Y))
    ax=(max_x_coord-min_x_coord)/nRow
    bx=min_x_coord
    ay=(max_y_coord-min_y_coord)/nCol
    by=min_y_coord

    #service data
    map_watersheds_accumulation_water=[]

    #check folders
    if(save_history==True):
         if not os.path.exists(histry_dir):
                os.makedirs(histry_dir)
         clouds_data=histry_dir+"\\"+sub_dir_clouds_history
         if not os.path.exists(clouds_data):
                os.makedirs(clouds_data)
         time_series_intake=histry_dir+"\\"+sub_dir_time_series_nodes_intake
         if not os.path.exists(time_series_intake):
             os.makedirs(time_series_intake)
         terrain_data=histry_dir+"\\"+sub_dir_terrain_hist
         if not os.path.exists(terrain_data):
             os.makedirs(terrain_data)
         terrain_water_accum=terrain_data+"\\Water_Accum_Per_WaterSheds"
         if not os.path.exists(terrain_water_accum):
             os.makedirs(terrain_water_accum)
         nodes_params=histry_dir+"\\"+sub_dir_nodes_params
         if not os.path.exists(nodes_params):
             os.makedirs(nodes_params)

    #simple coud, the water falls from cloud directly into the node without watersheds or any other interfaces inbwteen
    #pointwise water introduction
        
    if(scenario=="Clouds"):

        nCol=spatial_domain_nCol
        nRow=spatial_domain_nRows
        CLOUDS_GENERATOR_TYPE=rand_gen_type
        GRADIENT_TRANSITION_SIZE=clouds_grad_transition
        chaos_attr=None
        
        front_buf,chaos_attr = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr) 
        back_buf,chaos_attr  = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr) 
        back_buf=CorrectWithTransitionMap(GRADIENT_TRANSITION_SIZE, map_1 = front_buf, map_2 = back_buf)
        
        #start generation of the inputs
        cnt=0
        for i in range(0,time_series_length):
            #first we place the cloud over the spatial domain (e.g. over graph)
            
            if(cnt > nRow + GRADIENT_TRANSITION_SIZE):
                sz_front_buf=np.shape(front_buf)
                front_buf= back_buf[0:sz_front_buf[0]][:]
                back_buf,chaos_attr = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr) #nlmpy.mpd(nRow=50, nCol=50, h=0.75)
                back_buf=CorrectWithTransitionMap(GRADIENT_TRANSITION_SIZE, map_1 = front_buf, map_2 = back_buf)
                cnt=0
            mat_show=StickImages(step=cnt, map_1 = front_buf, map_2 = back_buf)
            cnt=cnt+1
            
            for j in range(0,len(nodes)):
                #original coordinates
                pos_x=nodes_X[j]
                pos_y=nodes_Y[j]    
                #tranlated into spatial domain 
                map_x=int((pos_x-bx)/ax)
                map_y=int((pos_y-by)/ay)

                if(map_x>=nRow):
                    map_x=nRow-1
                if(map_y>=nCol):
                    map_y=nCol-1

                rain_input_value=mat_show[map_x,map_y]
                rain_node_input[j].append(rain_input_value)

            if(save_history==True):
                #save mat_show               
                df = pd.DataFrame(mat_show)
                path_to_save_full=clouds_data+"\\cloud_time_step_"+str(i)+".csv"
                df.to_csv(path_to_save_full, header=False, index=False, sep=' ')

        #save nodes input history
        if(save_history==True):            
            #save nodes intake
            for j in range(0,len(nodes)):
                full_path_file=time_series_intake+"\\"+str(nodes[j])+"_rain_input.csv"
                df = pd.DataFrame(rain_node_input[j])
                df.to_csv(full_path_file, header=False, index=False, sep =' ')
                
        return rain_node_input

    if(scenario == "Clouds+terrain"):
        
        #terrains and its watersheds segmentations
        # watersheds maps can be gererated using watershed algorithm for segmentation
        terrain_map,tmp = Terrain_Gen("nlmpy",nRow=nRow, nCol=nCol, h=0.86)
        l1,l2=WatershedsSeparation(terrain_map,levels=4) #l1 is the small scale, and l2 is the merged version of l1 with levels, given by user
        
        if(save_history==True):
            #save as scv file
            path_terrain=terrain_data+"\\original_terrain.csv"
            df = pd.DataFrame(terrain_map)
            df.to_csv(path_terrain, header=False, index=False, sep =' ')
            #labbeled terrain with watersheds
            path_terrain_watershed=terrain_data+"\\watershed_retailed.csv"
            df = pd.DataFrame(l1)
            df.to_csv(path_terrain_watershed, header=False, index=False, sep =' ')
            #merged watershed
            path_terrain_watershed_leveled=terrain_data+"\\watershed_user_leveled.csv"
            df = pd.DataFrame(l2)
            df.to_csv(path_terrain_watershed_leveled, header=False, index=False, sep =' ')
            
            #save as picture
            path_terrain_figure=terrain_data+"\\terrain_pictures.jpg"
            fig3,(ax1,ax2,ax3) = plt.subplots(1,3)
            #ax1,ax2 = fig3.add_subplot(211)
            ax1.imshow(terrain_map)            
            ax2.imshow(l1, cmap='viridis', interpolation='nearest')            
            ax3.imshow(l2, cmap='viridis', interpolation='nearest')            
            #plt.show(block=False)
            plt.savefig(path_terrain_figure)
            plt.close()
            
        nCol=spatial_domain_nCol
        nRow=spatial_domain_nRows
        CLOUDS_GENERATOR_TYPE=rand_gen_type
        GRADIENT_TRANSITION_SIZE=clouds_grad_transition
        chaos_attr=None
            
        front_buf,chaos_attr = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr) 
        back_buf,chaos_attr  = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr) 
        back_buf=CorrectWithTransitionMap(GRADIENT_TRANSITION_SIZE, map_1 = front_buf, map_2 = back_buf)
            
        #start generation of the inputs
        cnt=0
        for sim_time_step in range(0,time_series_length):
                        
            #first we place the cloud over the spatial domain (e.g. over graph)
              
            if(cnt > nRow + GRADIENT_TRANSITION_SIZE):
                sz_front_buf=np.shape(front_buf)
                front_buf= back_buf[0:sz_front_buf[0]][:]
                back_buf,chaos_attr = Terrain_Gen(CLOUDS_GENERATOR_TYPE,nRow=nRow, nCol=nCol, h=h,chaos_obj = chaos_attr) #nlmpy.mpd(nRow=50, nCol=50, h=0.75)
                back_buf=CorrectWithTransitionMap(GRADIENT_TRANSITION_SIZE, map_1 = front_buf, map_2 = back_buf)
                cnt=0
                
            mat_show=StickImages(step=cnt, map_1 = front_buf, map_2 = back_buf)
            cnt=cnt+1
            
            #----------------------------------------------------------------------
            #compute how much rain went inside each segment (e.g. watershed)
            #----------------------------------------------------------------------
            labels=[]
            water_ccum=[]
            #unique_labels=np.unique(l1)
            for i in range(0,nRow):
                for j in range(0,nCol):
                    label=l1[i,j]
                    if (label in labels)==False:
                        labels.append(label)
                        water_ccum.append(mat_show[i,j])
                    else:
                        index = labels.index(label)
                        water_ccum[index]=water_ccum[index] + mat_show[i,j]

            if(save_history==True):
                map_watersheds_accumulation_water=np.zeros((nRow,nCol))
                for i in range(0,nRow):
                   for j in range(0,nCol):  
                       label=l1[i,j]
                       index = labels.index(label)
                       map_watersheds_accumulation_water[i,j] = water_ccum[index]
                #now we save this as CSV file
                df = pd.DataFrame(map_watersheds_accumulation_water)
                path_to_save_full=terrain_water_accum+"\\watershed_accum_map_time_stamp_"+str(sim_time_step)+".csv"
                df.to_csv(path_to_save_full, header=False, index=False, sep=' ')
        
            #----------------------------------------------------------------------
            #-------------------------------------------
            #----------------------------------------------------------------------
                
            for j in range(0,len(nodes)):
                 #original coordinates
                 pos_x=nodes_X[j]
                 pos_y=nodes_Y[j]    
                 #tranlated into spatial domain 
                 map_x=int((pos_x-bx)/ax)
                 map_y=int((pos_y-by)/ay)
    
                 if(map_x>=nRow):
                     map_x=nRow-1
                 if(map_y>=nCol):
                     map_y=nCol-1               
        
                 label=l1[map_x,map_y]
                 index = labels.index(label)
                 rain_input_value=water_ccum[index]
                 rain_node_input[j].append(rain_input_value)
    
            if(save_history==True):                 
                 #save mat_show  which is the current cloud             
                 df = pd.DataFrame(mat_show)
                 path_to_save_full=clouds_data+"\\cloud_time_step_"+str(sim_time_step)+".csv"
                 df.to_csv(path_to_save_full, header=False, index=False, sep=' ')
    
        #save nodes input history
        if(save_history==True):            
             #save nodes intake
             for j in range(0,len(nodes)):
                 full_path_file=time_series_intake+"\\"+str(nodes[j])+"_rain_input.csv"
                 df = pd.DataFrame(rain_node_input[j])
                 df.to_csv(full_path_file, header=False, index=False, sep =' ')
                
        return rain_node_input

In [ ]:
import UTwinRoutine as utr
fn="C:\\Users\\shse\\Documents\\ANACONDA\\UTwin_notebooks\\UTWIN_notebooks\\faf_feb16_apr20.inp"
L1,L2,L3,L4,L5,L6,L7,L8,L8_1,L8_2,L8_3,L8_4,L8_5,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21,L22,L23,L24,L25=utr.OpenInp(fn)
print(L8_2)

In [ ]:
import UTwinRoutine as utr
fn="C:\\Users\\shse\\Documents\\ANACONDA\\UTwin_notebooks\\UTWIN_notebooks\\faf_feb16_apr20.inp"
L1,L2,L3,L4,L5,L6,L7,L8,L8_1,L8_2,L8_3,L8_4,L8_5,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21,L22,L23,L24,L25=utr.OpenInp(fn)
Graph,points,pos2=MakeGraph(L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21,L22,L23,L24,L25)
ppModel,nodes_names,pipes_names=Pandapipes_makeNet(graph=Graph)

In [ ]:
import UTwinRoutine as utr
fn="C:\\Users\\shse\\Documents\\ANACONDA\\UTwin_notebooks\\UTWIN_notebooks\\faf_feb16_apr20.inp"
L1,L2,L3,L4,L5,L6,L7,L8,L8_1,L8_2,L8_3,L8_4,L8_5,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21,L22,L23,L24,L25=utr.OpenInp(fn)


Graph,points,pos2=MakeGraph(L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21,L22,L23,L24,L25)

Node_Names=L1
Nodes_coord_X=L2
Nodes_coord_Y=L3
sim_steps=2000
nRow=50
nCol=50
waterIntakeScenario="Clouds+terrain" # "clouds"

Sim_ub_dir_clouds_history="Clouds"
Sim_sub_dir_terrain_hist="Terrain"
Sim_sub_dir_time_series_nodes_intake="Nodes_intake"
Sim_ub_dir_nodes_params="Nodes_params"

now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
path_save_all_data="C:\Temp\WEATHER_GENERATOR\\"+dt_string

time_series=GenerateSpatialDomain1(scenario=waterIntakeScenario,
                                                 nodes=Node_Names,
                                                 time_series_length=sim_steps,
                                                 nodes_X=Nodes_coord_X,
                                                 nodes_Y=Nodes_coord_Y,
                                                 spatial_domain_nRows=nRow,
                                                 spatial_domain_nCol=nCol,
                                                 rand_gen_type="nlmpy",#"const","nlmpy", "lorenz_attractor", "white_noise","const"
                                                 clouds_grad_transition=0,
                                                 #if to save all the history
                                                 save_history=True,
                                                 histry_dir=path_save_all_data,
                                                 #this are the subdir names
                                                 sub_dir_clouds_history=Sim_ub_dir_clouds_history,
                                                 sub_dir_terrain_hist=Sim_sub_dir_terrain_hist,
                                                 sub_dir_time_series_nodes_intake=Sim_sub_dir_time_series_nodes_intake,
                                                 sub_dir_nodes_params=Sim_ub_dir_nodes_params
                                             )

self.WnModel,self.WNTR_results,self.WNTR_nodes,self.WNTR_links=WNTR1(graph=self.Graph,
                                                                            sim_dur=sim_steps,
                                                                            duration_type=sim_dur_type,
                                                                            useWaterInputs=True, 
                                                                            NodeNames=Node_Names, 
                                                                            PatternsTimeSerries=time_series
                                                                           )


In [ ]:
plt.plot(time_series[0])

In [ ]:
import UTwinRoutine as utr
fn="C:\\Users\\shse\\Documents\\ANACONDA\\UTwin_notebooks\\UTWIN_notebooks\\faf_feb16_apr20.inp"
L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21,L22,L23,L24,L25=utr.OpenInp(fn)
Graph,points,pos2=MakeGraph(L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21,L22,L23,L24,L25)
results,WNTR_nodes,WNTR_links=WNTR(graph=Graph,sim_dur=30000)                              
node_keys = results.node.keys()
link_keys = results.link.keys()

node_keys= list(node_keys)[:]
link_keys= list(link_keys)[:]
for i in node_keys:
    print(i)


pressure = results.node['pressure']
pressure_at_node123 =  results.node['pressure'].loc[:,'123']
fig = plt.figure()
ax = pressure_at_node123.plot()
text = ax.set_xlabel("Time (s)")
text = ax.set_ylabel("Pressure (m)")

In [ ]:
import wntr
wn = wntr.network.WaterNetworkModel(fn) 

In [ ]:
"""
self.Node_Names=L1
self.Nodes_coord_X=L2
self.Nodes_coord_Y=L3
self.Cond_Names=L4
self.Nodes_in=L5
self.Nodes_out=L6
self.Length=L7
self.Roughness=L8
self.Junc_Names=L9
self.Junc_elevation=L10
self.Junc_max_depth=L11
self.Junc_init_depth=L12
self.Storage_name=L13
self.Storage_elevation=L14
self.Storage_max_depth=L15
self.Pump_name=L16
self.Pump_From_Node=L17
self.Pump_To_Node=L18
self.Pump_Curve=L19
self.Pump_Status=L20
self.Pump_Sartup=L21
self.Pump_Shutoff=L22
self.Outfalls_name=L23
self.Outfalls_elevation=L24
self.Outfalls_type=L25
"""

In [ ]:
from pywr.core import Model, Input, Output

# create a model (including an empty network)
model = Model()

# create two nodes: a supply, and a demand
supply = Input(model, name='supply')
demand = Output(model, name='demand')

# create a connection from the supply to the demand
supply.connect(demand)
# set maximum flows
supply.max_flow = 10.0
demand.max_flow = 6.0
# set cost (+ve) or benefit (-ve)
supply.cost = 3.0
demand.cost = -100.0

import datetime
from pywr.core import Timestepper

model.timestepper = Timestepper(
    pandas.to_datetime('2015-01-01'),  # first day
    pandas.to_datetime('2015-12-31'),  # last day
    datetime.timedelta(1)  # interval
)
from pywr.recorders import NumpyArrayNodeRecorder

recorder = NumpyArrayNodeRecorder(model, supply)
# lets get this party started!
model.run()
scenario = 0
timestep = 0
#print(recorder.data[scenario][timestep])  # prints 6.0
print(recorder.data[scenario][:])